# Formateo de Datos
Como mencionamos anteriormente. Nos aprovecharemos de la forma de la base de datos para darle un formato más adecuado para su análisis. La estructura constará de usar un múlti indice en la parte de las columnas y de renglonres los municipios. 

## Extracción de Datos crudos.

In [1]:
import pandas as pd
import numpy as np

Sonora = pd.read_excel('..\\medium_data\\book_son.xlsx',sheet_name=None)
Municipios = list(Sonora.keys())

Ahora tomaremos los dataframes generados en la librera format_data.

In [5]:
from transform import indicators_tables
sonora = {municipio: indicators_tables.indicators_tables(Sonora, municipio) for municipio in Municipios}


def save_df(df: pd.DataFrame, filename: str):
    save_path = "..\\..\\results\\format_data\\"
    df.to_excel(save_path + filename)

ModuleNotFoundError: No module named 'transform'

## Construyendo las tablas.

Ahora iremos por cada DataFrame para ver su composición. Omitiremos la primera ya que se trata del nombre.

### Composición Territorial. (1)

In [279]:
mun = Municipios[0]
df = sonora[mun][1].copy()
df.set_index('Unnamed: 1',inplace=True)
df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
df.columns = df.iloc[0,:]
df = df.drop(index = ['Composición territorial','Fuente: INEGI.']).replace({'-' : 100}).astype(float)
df

Composición territorial,Absoluto,Porcentaje
Unnamed: 1,,
Localidades,25.0,100.0
Urbanas,1.0,4.0
Rurales,24.0,96.0
AGEB,14.0,100.0
Urbanas,11.0,78.6
Rurales,3.0,21.4
Manzanas,223.0,100.0
Urbanas,135.0,60.5
Rurales,88.0,39.5


Observamos que tenemos la Composición Territorial se conforma de 3 bloques principales. 
* Localidades
* AGEB
* Manzanas 
Las cuales a su vez se subdividen en Urbanas y Rurales. Cada una consta de dos datos. 
* Absoluto 
* Porcentaje. (Con respecto al total.)
* Total

Comenzaremos con la construcción de la tabla Base.

In [280]:
level_two = ['Absoluto','Porcentaje']
level_zero = ['Localidades', 'AGEB', 'Manzanas']
level_one = ['Total','Urbanas','Rurales']

lzero = len(level_zero)
lone = len(level_one)
ltwo = len(level_two)
ltotal = lzero*lone*ltwo

col_two = [two for j in range(lzero * lone)  for two in level_two ]
colone = [[one for one in level_one for j in range(ltwo)] for k in range(lzero)]
for l in range(lzero):
    if l == 0:
        col_one = colone[l]
    else:
        col_one += colone[l]
col_zero = [zero for zero in level_zero for j in range(lone * ltwo)]

cols = pd.MultiIndex.from_arrays([col_zero, col_one, col_two])
compos = pd.DataFrame(index = Municipios, columns=cols)
compos

Localidades                                 \
                                  Total             Urbanas              
                               Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                             NaN        NaN      NaN        NaN   
Agua Prieta                         NaN        NaN      NaN        NaN   
Alamos                              NaN        NaN      NaN        NaN   
Altar                               NaN        NaN      NaN        NaN   
Arivechi                            NaN        NaN      NaN        NaN   
...                                 ...        ...      ...        ...   
Villa Pesqueira                     NaN        NaN      NaN        NaN   
Yécora                              NaN        NaN      NaN        NaN   
Gral. Plutarco Elías Calles         NaN        NaN      NaN        NaN   
Benito Juárez                       NaN        NaN      NaN        NaN   
San Ignacio Río Muerto              NaN        NaN      NaN        NaN   

                                                    AGEB                      \
                             Rurales               Total             Urbanas   
                            Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                          NaN        NaN      NaN        NaN      NaN   
Agua Prieta                      NaN        NaN      NaN        NaN      NaN   
Alamos                           NaN        NaN      NaN        NaN      NaN   
Altar                            NaN        NaN      NaN        NaN      NaN   
Arivechi                         NaN        NaN      NaN        NaN      NaN   
...                              ...        ...      ...        ...      ...   
Villa Pesqueira                  NaN        NaN      NaN        NaN      NaN   
Yécora                           NaN        NaN      NaN        NaN      NaN   
Gral. Plutarco Elías Calles      NaN        NaN      NaN        NaN      NaN   
Benito Juárez                    NaN        NaN      NaN        NaN      NaN   
San Ignacio Río Muerto           NaN        NaN      NaN        NaN      NaN   

                                                           Manzanas  \
                                        Rurales               Total   
                            Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                            NaN      NaN        NaN      NaN   
Agua Prieta                        NaN      NaN        NaN      NaN   
Alamos                             NaN      NaN        NaN      NaN   
Altar                              NaN      NaN        NaN      NaN   
Arivechi                           NaN      NaN        NaN      NaN   
...                                ...      ...        ...      ...   
Villa Pesqueira                    NaN      NaN        NaN      NaN   
Yécora                             NaN      NaN        NaN      NaN   
Gral. Plutarco Elías Calles        NaN      NaN        NaN      NaN   
Benito Juárez                      NaN      NaN        NaN      NaN   
San Ignacio Río Muerto             NaN      NaN        NaN      NaN   

                                                                                
                                        Urbanas             Rurales             
                            Porcentaje Absoluto Porcentaje Absoluto Porcentaje  
Aconchi                            NaN      NaN        NaN      NaN        NaN  
Agua Prieta                        NaN      NaN        NaN      NaN        NaN  
Alamos                             NaN      NaN        NaN      NaN        NaN  
Altar                              NaN      NaN        NaN      NaN        NaN  
Arivechi                           NaN      NaN        NaN      NaN        NaN  
...                                ...      ...        ...      ...        ...  
Villa Pesqueira                    NaN      NaN        NaN      NaN        NaN  
Yécora                             NaN      NaN        NaN      N

Finalmente haremos el código para extraer y acomodar los datos correspondientes.

In [281]:
data_matrix = np.array(df)
absolute = data_matrix[:,0]
percent = data_matrix[:, 1]

data_mun = [absolute[i // 2] if i % 2 == 0 else percent[(i - 1) // 2] for i in range(2 * len(absolute))]

compos.iloc[0,:] = data_mun


compos

Localidades                                 \
                                  Total             Urbanas              
                               Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                            25.0      100.0      1.0        4.0   
Agua Prieta                         NaN        NaN      NaN        NaN   
Alamos                              NaN        NaN      NaN        NaN   
Altar                               NaN        NaN      NaN        NaN   
Arivechi                            NaN        NaN      NaN        NaN   
...                                 ...        ...      ...        ...   
Villa Pesqueira                     NaN        NaN      NaN        NaN   
Yécora                              NaN        NaN      NaN        NaN   
Gral. Plutarco Elías Calles         NaN        NaN      NaN        NaN   
Benito Juárez                       NaN        NaN      NaN        NaN   
San Ignacio Río Muerto              NaN        NaN      NaN        NaN   

                                                    AGEB                      \
                             Rurales               Total             Urbanas   
                            Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                         24.0       96.0     14.0      100.0     11.0   
Agua Prieta                      NaN        NaN      NaN        NaN      NaN   
Alamos                           NaN        NaN      NaN        NaN      NaN   
Altar                            NaN        NaN      NaN        NaN      NaN   
Arivechi                         NaN        NaN      NaN        NaN      NaN   
...                              ...        ...      ...        ...      ...   
Villa Pesqueira                  NaN        NaN      NaN        NaN      NaN   
Yécora                           NaN        NaN      NaN        NaN      NaN   
Gral. Plutarco Elías Calles      NaN        NaN      NaN        NaN      NaN   
Benito Juárez                    NaN        NaN      NaN        NaN      NaN   
San Ignacio Río Muerto           NaN        NaN      NaN        NaN      NaN   

                                                           Manzanas  \
                                        Rurales               Total   
                            Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                           78.6      3.0       21.4    223.0   
Agua Prieta                        NaN      NaN        NaN      NaN   
Alamos                             NaN      NaN        NaN      NaN   
Altar                              NaN      NaN        NaN      NaN   
Arivechi                           NaN      NaN        NaN      NaN   
...                                ...      ...        ...      ...   
Villa Pesqueira                    NaN      NaN        NaN      NaN   
Yécora                             NaN      NaN        NaN      NaN   
Gral. Plutarco Elías Calles        NaN      NaN        NaN      NaN   
Benito Juárez                      NaN      NaN        NaN      NaN   
San Ignacio Río Muerto             NaN      NaN        NaN      NaN   

                                                                                
                                        Urbanas             Rurales             
                            Porcentaje Absoluto Porcentaje Absoluto Porcentaje  
Aconchi                          100.0    135.0       60.5     88.0       39.5  
Agua Prieta                        NaN      NaN        NaN      NaN        NaN  
Alamos                             NaN      NaN        NaN      NaN        NaN  
Altar                              NaN      NaN        NaN      NaN        NaN  
Arivechi                           NaN      NaN        NaN      NaN        NaN  
...                                ...      ...        ...      ...        ...  
Villa Pesqueira                    NaN      NaN        NaN      NaN        NaN  
Yécora                             NaN      NaN        NaN      N

Finalmente, agrupamos en una función y aplicamos a todos los municipios. 

In [282]:
for i, mun in enumerate(Municipios):
    df = sonora[mun][1].copy()
    df.set_index('Unnamed: 1',inplace=True)
    df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
    df.columns = df.iloc[0,:]
    df = df.drop(index = ['Composición territorial','Fuente: INEGI.'])
    data_matrix = np.array(df)
    absolute = data_matrix[:,0]
    percent = data_matrix[:, 1]
    data_mun = [absolute[i // 2] if i % 2 == 0 else percent[(i - 1) // 2] for i in range(2 * len(absolute))]
    compos.iloc[i,:] = data_mun


Finalmente guardamos. 

In [283]:
save_df(compos, "ComposicionTerritorial.xlsx")

### Ubicación Geográfica (2)

In [284]:
df = sonora['San Miguel de Horcasitas'][2].copy()
df.set_index('Unnamed: 1',inplace=True)
df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
df.columns = df.iloc[0,:]
df = df.drop(index = ['Ubicación geográfica','Fuente: INEGI.'])
df

Ubicación geográfica,Absoluto,Porcentaje
Unnamed: 1,,
Coordenadas,Entre los paralelos 29° 16’ y 29° 41’ de latit...,-
Altitud,Entre 200 y 900 metros.,-
Colindancias,Colinda al norte con los municipios de Carbó y...,-
Superficie (Km2),1115.6,0.6
Densidad de población (habitantes/km2),9.6,-


##### Construcción de la Tabla

In [285]:
column_two = ['Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas',
               'Colindancias', 'Superficie', 'Superficie', 'Densidad de Poblacion']
subcolumn_two = ['Latitud', 'Latitud', 'Longitud', 'Longitud', 'Altitud', 'Altitud',
                 '','Km^2','', 'Hab/Km^2']
subsubcolumn_two = ['Min (°)', 'Max (°)','Min (°)', 'Max (°)','Min (m)', 'Max (m)',
                    '','Absoluto', 'Porcentaje','Absoluto']
cols = pd.MultiIndex.from_arrays([column_two, subcolumn_two, subsubcolumn_two])
geo_son = pd.DataFrame(columns=cols, index = Municipios)

geo_son.iloc[0, 6] = df.loc['Colindancias','Absoluto']
geo_son.iloc[0, 7] = df.loc['Superficie (Km2)','Absoluto']
geo_son.iloc[0, 8] = df.loc['Superficie (Km2)','Porcentaje']
geo_son.iloc[0, 9] = df.loc['Densidad de población (habitantes/km2)','Absoluto']
geo_son

Coordenadas                                   \
                                Latitud         Longitud         Altitud   
                                Min (°) Max (°)  Min (°) Max (°) Min (m)   
Aconchi                             NaN     NaN      NaN     NaN     NaN   
Agua Prieta                         NaN     NaN      NaN     NaN     NaN   
Alamos                              NaN     NaN      NaN     NaN     NaN   
Altar                               NaN     NaN      NaN     NaN     NaN   
Arivechi                            NaN     NaN      NaN     NaN     NaN   
...                                 ...     ...      ...     ...     ...   
Villa Pesqueira                     NaN     NaN      NaN     NaN     NaN   
Yécora                              NaN     NaN      NaN     NaN     NaN   
Gral. Plutarco Elías Calles         NaN     NaN      NaN     NaN     NaN   
Benito Juárez                       NaN     NaN      NaN     NaN     NaN   
San Ignacio Río Muerto              NaN     NaN      NaN     NaN     NaN   

                                     \
                                      
                            Max (m)   
Aconchi                         NaN   
Agua Prieta                     NaN   
Alamos                          NaN   
Altar                           NaN   
Arivechi                        NaN   
...                             ...   
Villa Pesqueira                 NaN   
Yécora                          NaN   
Gral. Plutarco Elías Calles     NaN   
Benito Juárez                   NaN   
San Ignacio Río Muerto          NaN   

                                                                  Colindancias  \
                                                                                 
                                                                                 
Aconchi                      Colinda al norte con los municipios de Carbó y...   
Agua Prieta                                                                NaN   
Alamos                                                                     NaN   
Altar                                                                      NaN   
Arivechi                                                                   NaN   
...                                                                        ...   
Villa Pesqueira                                                            NaN   
Yécora                                                                     NaN   
Gral. Plutarco Elías Calles                                                NaN   
Benito Juárez                                                              NaN   
San Ignacio Río Muerto                                                     NaN   

                            Superficie            Densidad de Poblacion  
                                  Km^2                         Hab/Km^2  
                              Absoluto Porcentaje              Absoluto  
Aconchi                         1115.6        0.6                   9.6  
Agua Prieta                        NaN        NaN                   NaN  
Alamos                             NaN        NaN                   NaN  
Altar                              NaN        NaN                   NaN  
Arivechi                           NaN        NaN                   NaN  
...                                ...        ...                   ...  
Villa Pesqueira                    NaN        NaN                   NaN  
Yécora                             NaN        NaN                   NaN  
Gral. Plutarco Elías Calles        NaN        NaN                   NaN  
Benito Juárez                      NaN        NaN                   NaN  
San Ignacio Río Muerto             NaN        NaN                   NaN  

[72 rows x 10 columns]

Ahora llenaremos los datos.

In [286]:
import regex as re

def get_altitud(altitud: str):
    number_pat = "\d+"
    nums = re.findall(number_pat, altitud)
    alt_values = [float(nums[0]),float(nums[1])]
    if len(nums) > 2:
        isdig = [len(x) for x in nums]
        one_index = isdig.index(1)
        new_num = nums[one_index] + nums[one_index + 1]
        alt_values = [float(nums[one_index - 1]), float(new_num)]
    alt_values.sort()
    return alt_values

def add_text_at_list(u : list, text: str):
    n = len(u)
    for i in range(n):
        u[i] = str(u[i]) + text

def grade_to_real(coord: str):
    number_pattern = "(\d+)°\s(\d+)’?"
    coord = re.findall(number_pattern, coord)[0]
    real_coords = 0
    for i in range(len(coord)):
        real_coords += float(coord[i]) * 60 ** (-i)
    return real_coords 

coords_sign = {
    'norte': 1,
    'sur' : -1,
    'este': -1,
    'oeste': 1
}

def get_latlong(latlong: str):
    latitud, longitud = latlong.split(";")[:2]
    coords_pattern = "\d+°\s\d+’?" # 
    direction_pattern = "norte|sur|este|oeste"
    lat_coords = re.findall(coords_pattern, latitud)
    lat_dir = re.findall(direction_pattern, latitud)[0]
    lon_coords = re.findall(coords_pattern, longitud)
    lon_dir = re.findall(direction_pattern, longitud)[0]
    lat1 = grade_to_real(lat_coords[0])
    lat2 = grade_to_real(lat_coords[1])
    lon1 = grade_to_real(lon_coords[0])
    lon2 = grade_to_real(lon_coords[1])
    latitud = [lat1 * coords_sign[lat_dir],lat2 * coords_sign[lat_dir]]
    latitud.sort()
    longitud = [lon1 * coords_sign[lon_dir], lon2 * coords_sign[lon_dir]]
    longitud.sort()
    return latitud + longitud

coords = df.loc['Coordenadas','Absoluto']
altura = df.loc['Altitud','Absoluto']
latlonalt = get_latlong(coords) + get_altitud(altura)
geo_son.iloc[0, :6] = latlonalt
geo_son

Coordenadas                                     \
                                Latitud               Longitud               
                                Min (°)    Max (°)     Min (°)     Max (°)   
Aconchi                       29.266667  29.683333  110.083333  110.566667   
Agua Prieta                         NaN        NaN         NaN         NaN   
Alamos                              NaN        NaN         NaN         NaN   
Altar                               NaN        NaN         NaN         NaN   
Arivechi                            NaN        NaN         NaN         NaN   
...                                 ...        ...         ...         ...   
Villa Pesqueira                     NaN        NaN         NaN         NaN   
Yécora                              NaN        NaN         NaN         NaN   
Gral. Plutarco Elías Calles         NaN        NaN         NaN         NaN   
Benito Juárez                       NaN        NaN         NaN         NaN   
San Ignacio Río Muerto              NaN        NaN         NaN         NaN   

                                             \
                            Altitud           
                            Min (m) Max (m)   
Aconchi                       200.0   900.0   
Agua Prieta                     NaN     NaN   
Alamos                          NaN     NaN   
Altar                           NaN     NaN   
Arivechi                        NaN     NaN   
...                             ...     ...   
Villa Pesqueira                 NaN     NaN   
Yécora                          NaN     NaN   
Gral. Plutarco Elías Calles     NaN     NaN   
Benito Juárez                   NaN     NaN   
San Ignacio Río Muerto          NaN     NaN   

                                                                  Colindancias  \
                                                                                 
                                                                                 
Aconchi                      Colinda al norte con los municipios de Carbó y...   
Agua Prieta                                                                NaN   
Alamos                                                                     NaN   
Altar                                                                      NaN   
Arivechi                                                                   NaN   
...                                                                        ...   
Villa Pesqueira                                                            NaN   
Yécora                                                                     NaN   
Gral. Plutarco Elías Calles                                                NaN   
Benito Juárez                                                              NaN   
San Ignacio Río Muerto                                                     NaN   

                            Superficie            Densidad de Poblacion  
                                  Km^2                         Hab/Km^2  
                              Absoluto Porcentaje              Absoluto  
Aconchi                         1115.6        0.6                   9.6  
Agua Prieta                        NaN        NaN                   NaN  
Alamos                             NaN        NaN                   NaN  
Altar                              NaN        NaN                   NaN  
Arivechi                           NaN        NaN                   NaN  
...                                ...        ...                   ...  
Villa Pesqueira                    NaN        NaN                   NaN  
Yécora                             NaN        NaN                   NaN  
Gral. Plutarco Elías Calles        NaN        NaN                   NaN  
Benito Juárez                      NaN        NaN                   NaN  
San Ignacio Río Muerto             NaN        NaN                   NaN  

[72 rows x 10 columns]

Ahora lo aplicamos para cada municipio.

In [287]:
def geo_ubication(i: int, mun: str, uf: pd.DataFrame):
    df = sonora[mun][2].copy()
    df.set_index('Unnamed: 1',inplace=True)
    df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
    df.columns = df.iloc[0,:]
    df = df.drop(index = ['Ubicación geográfica','Fuente: INEGI.'])
    uf.iloc[i, 6] = df.loc['Colindancias','Absoluto']
    uf.iloc[i, 7] = df.loc['Superficie (Km2)','Absoluto']
    uf.iloc[i, 8] = df.loc['Superficie (Km2)','Porcentaje']
    uf.iloc[i, 9] = df.loc['Densidad de población (habitantes/km2)','Absoluto']
    coords = df.loc['Coordenadas','Absoluto']
    altura = df.loc['Altitud','Absoluto']
    latlonalt = get_latlong(coords) + get_altitud(altura)
    uf.iloc[i, :6] = latlonalt
    # print(mun)
  

for m, mun in enumerate(Municipios):
    geo_ubication(m, mun, geo_son)
geo_son


Coordenadas                                     \
                                Latitud               Longitud               
                                Min (°)    Max (°)     Min (°)     Max (°)   
Aconchi                       29.683333  29.883333       110.0  110.433333   
Agua Prieta                        30.7      31.35  108.683333      109.75   
Alamos                        26.383333  27.783333  108.416667      109.25   
Altar                         30.516667  31.666667       111.6  112.283333   
Arivechi                      28.733333  28.983333  108.833333  109.266667   
...                                 ...        ...         ...         ...   
Villa Pesqueira               28.783333       29.5  109.783333  110.133333   
Yécora                        28.166667      28.65  108.466667  110.133333   
Gral. Plutarco Elías Calles   31.316667  32.083333       112.3  113.516667   
Benito Juárez                 26.966667  27.233333  109.716667  109.983333   
San Ignacio Río Muerto        27.116667  27.533333  110.133333       110.6   

                                             \
                            Altitud           
                            Min (m) Max (m)   
Aconchi                       500.0  2200.0   
Agua Prieta                   800.0  2500.0   
Alamos                         50.0  2000.0   
Altar                         200.0  1700.0   
Arivechi                      400.0  2200.0   
...                             ...     ...   
Villa Pesqueira                50.0  1700.0   
Yécora                        300.0  2300.0   
Gral. Plutarco Elías Calles   100.0  1400.0   
Benito Juárez                   0.0   100.0   
San Ignacio Río Muerto          0.0   100.0   

                                                                  Colindancias  \
                                                                                 
                                                                                 
Aconchi                      Colinda al norte con los municipios de San Fel...   
Agua Prieta                  Colinda al norte con los Estados Unidos de Amé...   
Alamos                       Colinda al norte con el municipio de Quiriego ...   
Altar                        Colinda al norte con el municipio de Caborca y...   
Arivechi                     Colinda al norte, este, sur y oeste con el mun...   
...                                                                        ...   
Villa Pesqueira              Colinda al norte con los municipios de Baviáco...   
Yécora                       Colinda al norte con los municipios de Bacanor...   
Gral. Plutarco Elías Calles  Colinda al norte con el municipio de Puerto Pe...   
Benito Juárez                Colinda al norte con los municipios de Cajeme ...   
San Ignacio Río Muerto       Colinda al norte con los municipios de Guaymas...   

                            Superficie            Densidad de Poblacion  
                                  Km^2                         Hab/Km^2  
                              Absoluto Porcentaje              Absoluto  
Aconchi                          368.2        0.2                     7  
Agua Prieta                     3946.5        2.2                  23.3  
Alamos                          6422.8        3.6                   3.9  
Altar                           4457.7        2.5                   2.1  
Arivechi                         726.3        0.4                   1.6  
...                                ...        ...                   ...  
Villa Pesqueira                 1123.2        0.6                   0.9  
Yécora                          2667.7        1.5                   1.8  
Gral. Plutarco Elías Calles     3656.7          2                   3.7  
Benito Juárez                    369.5        0.2                  58.7  
San Ignacio Río Muerto          1383.6        0.8                  10.3  

[72 rows x 10 columns]

Ahora guardamos

In [288]:
save_df(geo_son, "UbicacionGeografica.xlsx")

### Dinamica Poblacional. (3/4)
En este caso, trabajaremos con dos tablas de forma simultanea.

#### Extracción de Datos
Tabla 1

In [289]:
mun = Municipios[31]
df = sonora[mun][3].copy()
df.set_index('Unnamed: 1',inplace=True)
df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
df.columns = df.iloc[0,:]
pob1 = df.drop(index = ['Dinámica poblacional (2020)'])
if len(df.columns) != 2:
    pob1 = pob1.iloc[:, :-1]


Tabla 2

In [290]:
df2 = sonora[mun][4].copy()
df2.set_index('Unnamed: 1',inplace=True)
df2 = df2.drop(index = 'nan', axis = 1)
df2 = df2.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all')
df2.columns = df2.iloc[0, :]
df2 = df2.drop(index = ['Dinámica poblacional (2020)'], axis = 0)

feet_pattern = '^\d'
inegi = 'INEGI'

index = list(df2.index)

extra_poblation = []  # Información Aidicional
for i in index:
    if re.findall(feet_pattern, i):
        extra_poblation.append(i)
        df2 = df2.drop(index = i)
    if re.findall(inegi, i):
        df2 = df2.drop(index = i)
extra_poblation[0] += 'y la otra mitad por encima.'
extra_poblation[1] += '(15 a 64 años)'
pob2 = df2.iloc[:-2,:2]
pob2


Dinámica poblacional (2020),Absoluto,Porcentaje
Unnamed: 1,,
Población masculina,467,52.6
Población femenina,421,47.4
Población de 0 a 14 años,178,20
Población de 15 a 64 años,550,61.9
Población de 65 años y más,160,18
Población en localidades de 1 a 2 499 habitantes,888,100
Población en localidades de 2 500 y más habitantes,0,0
Edad mediana 1,41,-
Razón de dependencia 2,61.454545,-


Una vez que tenemos ambos dataframes. Comenzamos con la creación de la tabla.

#### Creación de la Tabla

Por el número de columnas que vamos a necesitar las haremos por separado.

##### Población Total

In [291]:
c0 = ['Población Total','Tasa de Crecimiento Poblacional', 'Indice', 'Migración']
c1_pob = ['Año', 'Sexo', 'Edad', 'Localidades']
c1_values = {
    'Año' : ['1990','2000','2010','2020'],
    'Sexo': ['Masculino', 'Fememino'],
    'Edad': ['0-14', '15-64','65+'],
    'Localidades': ['1-2499','+2500']
}
clast = ['Absoluto', 'Porcentaje']

pob_two = [c for cat in c1_pob for c in c1_values[cat] for j in range(len(clast))]
pob_one = [c1 for c1 in c1_pob for j in range(len(c1_values[c1]) * len(clast))]
pob_three = [c for j in range(len(pob_two) // 2) for c in clast ]
pob_zero = [c0[0] for j in range(len(pob_two))]

mi_pob = pd.MultiIndex.from_arrays([pob_zero, pob_one, pob_two, pob_three])

df_pob = pd.DataFrame(index = Municipios, columns=mi_pob)
df_pob.head()


Población Total                                          \
                        Año                                           
                       1990                2000                2010   
                   Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                 NaN        NaN      NaN        NaN      NaN   
Agua Prieta             NaN        NaN      NaN        NaN      NaN   
Alamos                  NaN        NaN      NaN        NaN      NaN   
Altar                   NaN        NaN      NaN        NaN      NaN   
Arivechi                NaN        NaN      NaN        NaN      NaN   

                                                                 ...           \
                                                Sexo             ...     Edad   
                           2020            Masculino             ...     0-14   
            Porcentaje Absoluto Porcentaje  Absoluto Porcentaje  ... Absoluto   
Aconchi            NaN      NaN        NaN       NaN        NaN  ...      NaN   
Agua Prieta        NaN      NaN        NaN       NaN        NaN  ...      NaN   
Alamos             NaN      NaN        NaN       NaN        NaN  ...      NaN   
Altar              NaN      NaN        NaN       NaN        NaN  ...      NaN   
Arivechi           NaN      NaN        NaN       NaN        NaN  ...      NaN   

                                                                            \
                                                               Localidades   
                          15-64                 65+                 1-2499   
            Porcentaje Absoluto Porcentaje Absoluto Porcentaje    Absoluto   
Aconchi            NaN      NaN        NaN      NaN        NaN         NaN   
Agua Prieta        NaN      NaN        NaN      NaN        NaN         NaN   
Alamos             NaN      NaN        NaN      NaN        NaN         NaN   
Altar              NaN      NaN        NaN      NaN        NaN         NaN   
Arivechi           NaN      NaN        NaN      NaN        NaN         NaN   

                                            
                                            
                          +2500             
            Porcentaje Absoluto Porcentaje  
Aconchi            NaN      NaN        NaN  
Agua Prieta        NaN      NaN        NaN  
Alamos             NaN      NaN        NaN  
Altar              NaN      NaN        NaN  
Arivechi           NaN      NaN        NaN  

[5 rows x 22 columns]

#### Tasa de Crecimiento Poblacional.

In [292]:
c2_one = ['Periodo']
c2_two = ['1990-2000','2000-2010', '2010-2020']

tcp_two = [c2  for c2 in c2_two for j in range(len(clast))]
tcp_one = [c2 for c2 in c2_one for j in range(len(tcp_two))]
tcp_three = [c for j in range(len(tcp_two) // 2) for c in clast ]
tcp_zero = [c0[1] for j in range(len(tcp_two))]

mi_tcp = pd.MultiIndex.from_arrays([tcp_zero, tcp_one, tcp_two, tcp_three])
df_tcp = pd.DataFrame(index = Municipios, columns=mi_tcp)
df_tcp.head()

Tasa de Crecimiento Poblacional                                  \
                                    Periodo                                   
                                  1990-2000            2000-2010              
                                   Absoluto Porcentaje  Absoluto Porcentaje   
Aconchi                                 NaN        NaN       NaN        NaN   
Agua Prieta                             NaN        NaN       NaN        NaN   
Alamos                                  NaN        NaN       NaN        NaN   
Altar                                   NaN        NaN       NaN        NaN   
Arivechi                                NaN        NaN       NaN        NaN   

                                  
                                  
            2010-2020             
             Absoluto Porcentaje  
Aconchi           NaN        NaN  
Agua Prieta       NaN        NaN  
Alamos            NaN        NaN  
Altar             NaN        NaN  
Arivechi          NaN        NaN

#### Indice

In [293]:
c1_ind = ['Edad mediana', 'Edad mediana',
          'Razón de dependencia', 'Razón de dependencia',
          'Índice de envejecimiento', 'Índice de envejecimiento', 
          'Promedio de hijas e hijos nacidos vivos por mujer',
            'Promedio de hijas e hijos nacidos vivos por mujer']
c2_ind = ['(1)', '(1)', '(2)', '(2)', '(3)', '(3)', '(4)', '(4)']
c3_ind = ['Absoluto','Porcentaje',
          'Absoluto','Porcentaje',
          'Absoluto','Porcentaje',
          'Absoluto','Porcentaje',]
indice_zero = [c0[2] for i in range(len(c1_ind))]

mi_ind = pd.MultiIndex.from_arrays([indice_zero, c1_ind, c2_ind, c3_ind])
df_ind = pd.DataFrame(index = Municipios, columns = mi_ind)
df_ind.head()


Indice                                             \
            Edad mediana            Razón de dependencia              
                     (1)                             (2)              
                Absoluto Porcentaje             Absoluto Porcentaje   
Aconchi              NaN        NaN                  NaN        NaN   
Agua Prieta          NaN        NaN                  NaN        NaN   
Alamos               NaN        NaN                  NaN        NaN   
Altar                NaN        NaN                  NaN        NaN   
Arivechi             NaN        NaN                  NaN        NaN   

                                                 \
            Índice de envejecimiento              
                                 (3)              
                            Absoluto Porcentaje   
Aconchi                          NaN        NaN   
Agua Prieta                      NaN        NaN   
Alamos                           NaN        NaN   
Altar                            NaN        NaN   
Arivechi                         NaN        NaN   

                                                                          
            Promedio de hijas e hijos nacidos vivos por mujer             
                                                          (4)             
                                                     Absoluto Porcentaje  
Aconchi                                                   NaN        NaN  
Agua Prieta                                               NaN        NaN  
Alamos                                                    NaN        NaN  
Altar                                                     NaN        NaN  
Arivechi                                                  NaN        NaN

#### Migración

Finalmente, la útima categoría.

In [294]:
c1_migra = ['Total', 'Causa']
c2_migra = ['Trabajo', 'Asuntos Familiares', 'Estudio', 'Inseguridad delictiva o Violencia', 'Otro']

migra_three = [three  for j in range(len(c2_migra) + 1) for three in clast]
migra_two_1 = ["",""]
migra_two_2 = [two for two in c2_migra for j in range(len(clast))]
migra_two = migra_two_1 + migra_two_2
migra_one_1 = [one for one in c1_migra for j in range(len(clast))]
migra_one_2 = [c1_migra[1] for j in range(2 * (len(c2_migra) - 1))]
migra_one = migra_one_1 + migra_one_2
migra_zero = [c0[3] for i in range(len(migra_one))]


mi_migra = pd.MultiIndex.from_arrays([migra_zero, migra_one, migra_two, migra_three])
df_migra = pd.DataFrame(index = Municipios, columns=mi_migra)
df_migra

Migración                                 \
                                Total               Causa              
                                                  Trabajo              
                             Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                           NaN        NaN      NaN        NaN   
Agua Prieta                       NaN        NaN      NaN        NaN   
Alamos                            NaN        NaN      NaN        NaN   
Altar                             NaN        NaN      NaN        NaN   
Arivechi                          NaN        NaN      NaN        NaN   
...                               ...        ...      ...        ...   
Villa Pesqueira                   NaN        NaN      NaN        NaN   
Yécora                            NaN        NaN      NaN        NaN   
Gral. Plutarco Elías Calles       NaN        NaN      NaN        NaN   
Benito Juárez                     NaN        NaN      NaN        NaN   
San Ignacio Río Muerto            NaN        NaN      NaN        NaN   

                                                                               \
                                                                                
                            Asuntos Familiares             Estudio              
                                      Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                                    NaN        NaN      NaN        NaN   
Agua Prieta                                NaN        NaN      NaN        NaN   
Alamos                                     NaN        NaN      NaN        NaN   
Altar                                      NaN        NaN      NaN        NaN   
Arivechi                                   NaN        NaN      NaN        NaN   
...                                        ...        ...      ...        ...   
Villa Pesqueira                            NaN        NaN      NaN        NaN   
Yécora                                     NaN        NaN      NaN        NaN   
Gral. Plutarco Elías Calles                NaN        NaN      NaN        NaN   
Benito Juárez                              NaN        NaN      NaN        NaN   
San Ignacio Río Muerto                     NaN        NaN      NaN        NaN   

                                                                          \
                                                                           
                            Inseguridad delictiva o Violencia              
                                                     Absoluto Porcentaje   
Aconchi                                                   NaN        NaN   
Agua Prieta                                               NaN        NaN   
Alamos                                                    NaN        NaN   
Altar                                                     NaN        NaN   
Arivechi                                                  NaN        NaN   
...                                                       ...        ...   
Villa Pesqueira                                           NaN        NaN   
Yécora                                                    NaN        NaN   
Gral. Plutarco Elías Calles                               NaN        NaN   
Benito Juárez                                             NaN        NaN   
San Ignacio Río Muerto                                    NaN        NaN   

                                                 
                                                 
                                Otro             
                            Absoluto Porcentaje  
Aconchi                          NaN        NaN  
Agua Prieta                      NaN        NaN  
Alamos                           NaN        NaN  
Altar                            NaN        NaN  
Arivechi                         NaN        NaN  
...                              ...        ...  
Villa Pesqueira                  NaN        NaN  
Yécora                           NaN    

Entonces comenzamos con la extracción de los datos. Para hacerlo más ligero de visualizar, haremos esto por bloques.

#### Bloque 1: Población Total

Primero definiremos algunas listas donde guardar la información.

In [295]:
def dataframe_tolist(data: pd.DataFrame):
    y = []
    for i in list(data.index):
        for j in list(data.columns):
            y.append(data.loc[i,j])
    return y

#   Poblacion por Año
pob2020 = pob1.iloc[:4]

anual_pob = []
for i in list(pob2020.index):
    for j in list(pob2020.columns):
        anual_pob.append(pob2020.loc[i,j])

df_pob.iloc[0,:8] = anual_pob

# Poblacion por Sexo

gender = pob2.iloc[:2,:]
gender_data = dataframe_tolist(gender)
df_pob.iloc[0, 8:12] = gender_data

# Población por Edad

age = pob2.iloc[2:5,:]
age_data = dataframe_tolist(age)
df_pob.iloc[0, 12:18] = age_data

# Poblacion por Localidad

localidad = pob2.iloc[5:7,:]
localidad_data = dataframe_tolist(localidad)
df_pob.iloc[0, 18:22] = localidad_data



El código anterior puede reescribirse como sigue

In [296]:
def pob2_to_df_pob(i: int, pob2: pd.DataFrame):
    index_pob2 = [0,2,5,7]
    index_dfpob = [8,12,18,22]
    for j in range(len(index_pob2) - 1):
        data = pob2.iloc[index_pob2[j] : index_pob2[j + 1], :]
        data_l = dataframe_tolist(data)
        df_pob.iloc[i, index_dfpob[j] : index_dfpob[j + 1]] = data_l


Ahora, se lo aplicamos a todos los municipios. Primero creamos el DataFrame correspondiente.

In [297]:
c0 = ['Población Total','Tasa de Crecimiento Poblacional', 'Indice', 'Migración']
c1_pob = ['Año', 'Sexo', 'Edad', 'Localidades']
c1_values = {
    'Año' : ['1990','2000','2010','2020'],
    'Sexo': ['Masculino', 'Fememino'],
    'Edad': ['0-14', '15-64','65+'],
    'Localidades': ['1-2499','+2500']
}
clast = ['Absoluto', 'Porcentaje']

pob_two = [c for cat in c1_pob for c in c1_values[cat] for j in range(len(clast))]
pob_one = [c1 for c1 in c1_pob for j in range(len(c1_values[c1]) * len(clast))]
pob_three = [c for j in range(len(pob_two) // 2) for c in clast ]
pob_zero = [c0[0] for j in range(len(pob_two))]

mi_pob = pd.MultiIndex.from_arrays([pob_zero, pob_one, pob_two, pob_three])
df_pob = pd.DataFrame(index = Municipios, columns=mi_pob)
df_pob.head()

Población Total                                          \
                        Año                                           
                       1990                2000                2010   
                   Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                 NaN        NaN      NaN        NaN      NaN   
Agua Prieta             NaN        NaN      NaN        NaN      NaN   
Alamos                  NaN        NaN      NaN        NaN      NaN   
Altar                   NaN        NaN      NaN        NaN      NaN   
Arivechi                NaN        NaN      NaN        NaN      NaN   

                                                                 ...           \
                                                Sexo             ...     Edad   
                           2020            Masculino             ...     0-14   
            Porcentaje Absoluto Porcentaje  Absoluto Porcentaje  ... Absoluto   
Aconchi            NaN      NaN        NaN       NaN        NaN  ...      NaN   
Agua Prieta        NaN      NaN        NaN       NaN        NaN  ...      NaN   
Alamos             NaN      NaN        NaN       NaN        NaN  ...      NaN   
Altar              NaN      NaN        NaN       NaN        NaN  ...      NaN   
Arivechi           NaN      NaN        NaN       NaN        NaN  ...      NaN   

                                                                            \
                                                               Localidades   
                          15-64                 65+                 1-2499   
            Porcentaje Absoluto Porcentaje Absoluto Porcentaje    Absoluto   
Aconchi            NaN      NaN        NaN      NaN        NaN         NaN   
Agua Prieta        NaN      NaN        NaN      NaN        NaN         NaN   
Alamos             NaN      NaN        NaN      NaN        NaN         NaN   
Altar              NaN      NaN        NaN      NaN        NaN         NaN   
Arivechi           NaN      NaN        NaN      NaN        NaN         NaN   

                                            
                                            
                          +2500             
            Porcentaje Absoluto Porcentaje  
Aconchi            NaN      NaN        NaN  
Agua Prieta        NaN      NaN        NaN  
Alamos             NaN      NaN        NaN  
Altar              NaN      NaN        NaN  
Arivechi           NaN      NaN        NaN  

[5 rows x 22 columns]

In [298]:
feet_pattern = '^\d'
inegi = 'INEGI'

for m, mun in enumerate(Municipios):
    # Construimos pob1
    df = sonora[mun][3].copy()
    df.set_index('Unnamed: 1',inplace=True)
    df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
    df.columns = df.iloc[0,:]
    pob1 = df.drop(index = ['Dinámica poblacional (2020)'])
    if len(df.columns) != 2:
        pob1 = pob1.iloc[:, :-1]
    
    # Construimos pob2
    df2 = sonora[mun][4].copy()
    df2.set_index('Unnamed: 1',inplace=True)
    df2 = df2.drop(index = 'nan', axis = 1)
    df2 = df2.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all')
    df2.columns = df2.iloc[0, :]
    df2 = df2.drop(index = ['Dinámica poblacional (2020)'], axis = 0)
    index = list(df2.index)
    extra_poblation = []  # Información Aidicional
    for i in index:
        if re.findall(feet_pattern, i):
            if m == 0:
                extra_poblation.append(i)
            df2 = df2.drop(index = i)
        if re.findall(inegi, i):
            df2 = df2.drop(index = i)
    if m == 0:
        extra_poblation[0] += 'y la otra mitad por encima.'
        extra_poblation[1] += '(15 a 64 años)'
    pob2 = df2.iloc[:-2,:2]
    # Año
    pobm = pob1.iloc[:4]
    anual_pob = []
    for i in list(pobm.index):
        for j in list(pobm.columns):
            anual_pob.append(pobm.loc[i,j])
    df_pob.iloc[m,:8] = anual_pob
    pob2_to_df_pob(m, pob2)    

Finalmente, tenemos el Bloque 1

In [299]:
df_pob.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, Aconchi to San Ignacio Río Muerto
Data columns (total 22 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   (Población Total, Año, 1990, Absoluto)              70 non-null     object
 1   (Población Total, Año, 1990, Porcentaje)            70 non-null     object
 2   (Población Total, Año, 2000, Absoluto)              72 non-null     object
 3   (Población Total, Año, 2000, Porcentaje)            72 non-null     object
 4   (Población Total, Año, 2010, Absoluto)              72 non-null     object
 5   (Población Total, Año, 2010, Porcentaje)            72 non-null     object
 6   (Población Total, Año, 2020, Absoluto)              72 non-null     object
 7   (Población Total, Año, 2020, Porcentaje)            72 non-null     object
 8   (Población Total, Sexo, Masculino, Absoluto)        72 non-null     obj

Bloque 2: Tasa de Crecimiento Poblacional

In [300]:
df_tcp.head()

Tasa de Crecimiento Poblacional                                  \
                                    Periodo                                   
                                  1990-2000            2000-2010              
                                   Absoluto Porcentaje  Absoluto Porcentaje   
Aconchi                                 NaN        NaN       NaN        NaN   
Agua Prieta                             NaN        NaN       NaN        NaN   
Alamos                                  NaN        NaN       NaN        NaN   
Altar                                   NaN        NaN       NaN        NaN   
Arivechi                                NaN        NaN       NaN        NaN   

                                  
                                  
            2010-2020             
             Absoluto Porcentaje  
Aconchi           NaN        NaN  
Agua Prieta       NaN        NaN  
Alamos            NaN        NaN  
Altar             NaN        NaN  
Arivechi          NaN        NaN

Ahora, debemos llenar la tabla. Lo conveniente que podemos tomar la construcción de pob1, y pob2 del caso anterior.

In [301]:
tcp1 = pob1[4:]
tc1p = dataframe_tolist(tcp1)
df_tcp.iloc[0] = tc1p
df_tcp

Tasa de Crecimiento Poblacional             \
                                                    Periodo              
                                                  1990-2000              
                                                   Absoluto Porcentaje   
Aconchi                                                   -          0   
Agua Prieta                                             NaN        NaN   
Alamos                                                  NaN        NaN   
Altar                                                   NaN        NaN   
Arivechi                                                NaN        NaN   
...                                                     ...        ...   
Villa Pesqueira                                         NaN        NaN   
Yécora                                                  NaN        NaN   
Gral. Plutarco Elías Calles                             NaN        NaN   
Benito Juárez                                           NaN        NaN   
San Ignacio Río Muerto                                  NaN        NaN   

                                                                       
                                                                       
                            2000-2010            2010-2020             
                             Absoluto Porcentaje  Absoluto Porcentaje  
Aconchi                             -        0.3         -        0.1  
Agua Prieta                       NaN        NaN       NaN        NaN  
Alamos                            NaN        NaN       NaN        NaN  
Altar                             NaN        NaN       NaN        NaN  
Arivechi                          NaN        NaN       NaN        NaN  
...                               ...        ...       ...        ...  
Villa Pesqueira                   NaN        NaN       NaN        NaN  
Yécora                            NaN        NaN       NaN        NaN  
Gral. Plutarco Elías Calles       NaN        NaN       NaN        NaN  
Benito Juárez                     NaN        NaN       NaN        NaN  
San Ignacio Río Muerto            NaN        NaN       NaN        NaN  

[72 rows x 6 columns]

Repetimos para cada municipio.

In [302]:
for m, mun in enumerate(Municipios):
    # Construimos pob1
    df = sonora[mun][3].copy()
    df.set_index('Unnamed: 1',inplace=True)
    df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
    df.columns = df.iloc[0,:]
    pob1 = df.drop(index = ['Dinámica poblacional (2020)'])
    pob1 = pob1.iloc[:, :2]
    tcp1 = pob1[4:]
    tc1p = dataframe_tolist(tcp1)
    df_tcp.iloc[m] = tc1p
df_tcp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, Aconchi to San Ignacio Río Muerto
Data columns (total 6 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   (Tasa de Crecimiento Poblacional, Periodo, 1990-2000, Absoluto)    72 non-null     object
 1   (Tasa de Crecimiento Poblacional, Periodo, 1990-2000, Porcentaje)  72 non-null     object
 2   (Tasa de Crecimiento Poblacional, Periodo, 2000-2010, Absoluto)    72 non-null     object
 3   (Tasa de Crecimiento Poblacional, Periodo, 2000-2010, Porcentaje)  72 non-null     object
 4   (Tasa de Crecimiento Poblacional, Periodo, 2010-2020, Absoluto)    72 non-null     object
 5   (Tasa de Crecimiento Poblacional, Periodo, 2010-2020, Porcentaje)  72 non-null     object
dtypes: object(6)
memory usage: 3.9+ KB


#### Bloque 3: Indice

In [303]:
for m,mun in enumerate(Municipios):
    df2 = sonora[mun][4].copy()
    df2.set_index('Unnamed: 1',inplace=True)
    df2 = df2.drop(index = 'nan', axis = 1)
    df2 = df2.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all')
    df2.columns = df2.iloc[0, :]
    df2 = df2.drop(index = ['Dinámica poblacional (2020)'], axis = 0)
    index = list(df2.index)
    extra_poblation = []  # Información Aidicional
    for i in index:
        if re.findall(feet_pattern, i):
            if m == 0:
                extra_poblation.append(i)
            df2 = df2.drop(index = i)
        if re.findall(inegi, i):
            df2 = df2.drop(index = i)
    if m == 0:
        extra_poblation[0] += 'y la otra mitad por encima.'
        extra_poblation[1] += '(15 a 64 años)'
    pob2 = df2.iloc[:-2,:2]
    indices = pob2.iloc[7:11,:]
    indic = dataframe_tolist(indices)
    df_ind.iloc[m,:] = indic
df_ind


Indice                                  \
                            Edad mediana            Razón de dependencia   
                                     (1)                             (2)   
                                Absoluto Porcentaje             Absoluto   
Aconchi                               34          -            57.820197   
Agua Prieta                           27          -            49.812478   
Alamos                                32          -            60.701416   
Altar                                 26          -            56.619346   
Arivechi                              38          -            66.478076   
...                                  ...        ...                  ...   
Villa Pesqueira                       44          -             60.95679   
Yécora                                32          -            64.849948   
Gral. Plutarco Elías Calles           27          -            59.922544   
Benito Juárez                         30          -            56.575718   
San Ignacio Río Muerto                29          -            55.189653   

                                                                            \
                                       Índice de envejecimiento              
                                                            (3)              
                            Porcentaje                 Absoluto Porcentaje   
Aconchi                              -                79.796265          -   
Agua Prieta                          -                33.194922          -   
Alamos                               -                 68.79634          -   
Altar                                -                35.174419          -   
Arivechi                             -                91.872792          -   
...                                ...                      ...        ...   
Villa Pesqueira                      -                165.19337          -   
Yécora                               -                63.685848          -   
Gral. Plutarco Elías Calles          -                34.245917          -   
Benito Juárez                        -                56.286472          -   
San Ignacio Río Muerto               -                53.239136          -   

                                                                               \
                            Promedio de hijas e hijos nacidos vivos por mujer   
                                                                          (4)   
                                                                     Absoluto   
Aconchi                                                              2.430542   
Agua Prieta                                                           2.15826   
Alamos                                                                2.62393   
Altar                                                                2.304513   
Arivechi                                                             2.595388   
...                                                                       ...   
Villa Pesqueira                                                      2.808081   
Yécora                                                               2.608231   
Gral. Plutarco Elías Calles                                          2.399272   
Benito Juárez                                                        2.370821   
San Ignacio Río Muerto                                               2.454282   

                                        
                                        
                                        
                            Porcentaje  
Aconchi                              -  
Agua Prieta                          -  
Alamos                               -  
Altar                                -  
Arivechi                             -  
...                                ...  
Villa Pesqueira                      -  
Yécora                               -  
Gral. Plutarco

Bloque 4: Migración. 

En este caso, ocuparemos hacer algo diferente por la categorización establecida

In [304]:
df_migra.head()


Migración                                                    \
                Total               Causa                                 
                                  Trabajo            Asuntos Familiares   
             Absoluto Porcentaje Absoluto Porcentaje           Absoluto   
Aconchi           NaN        NaN      NaN        NaN                NaN   
Agua Prieta       NaN        NaN      NaN        NaN                NaN   
Alamos            NaN        NaN      NaN        NaN                NaN   
Altar             NaN        NaN      NaN        NaN                NaN   
Arivechi          NaN        NaN      NaN        NaN                NaN   

                                                                              \
                                                                               
                        Estudio            Inseguridad delictiva o Violencia   
            Porcentaje Absoluto Porcentaje                          Absoluto   
Aconchi            NaN      NaN        NaN                               NaN   
Agua Prieta        NaN      NaN        NaN                               NaN   
Alamos             NaN      NaN        NaN                               NaN   
Altar              NaN      NaN        NaN                               NaN   
Arivechi           NaN      NaN        NaN                               NaN   

                                            
                                            
                           Otro             
            Porcentaje Absoluto Porcentaje  
Aconchi            NaN      NaN        NaN  
Agua Prieta        NaN      NaN        NaN  
Alamos             NaN      NaN        NaN  
Altar              NaN      NaN        NaN  
Arivechi           NaN      NaN        NaN

In [305]:
migration = pob2.loc['Población de 5 años y más migrante':]

migration_total = migration.iloc[0]
migration_causes = migration.iloc[2:,:]

migra_total = migration_total.tolist()
migra_causes = dataframe_tolist(migration_causes)

df_migra.iloc[0,:2] = migra_total
df_migra.iloc[0,2:] = migra_causes
df_migra

Migración                                 \
                                Total               Causa              
                                                  Trabajo              
                             Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                           445        3.4       98         22   
Agua Prieta                       NaN        NaN      NaN        NaN   
Alamos                            NaN        NaN      NaN        NaN   
Altar                             NaN        NaN      NaN        NaN   
Arivechi                          NaN        NaN      NaN        NaN   
...                               ...        ...      ...        ...   
Villa Pesqueira                   NaN        NaN      NaN        NaN   
Yécora                            NaN        NaN      NaN        NaN   
Gral. Plutarco Elías Calles       NaN        NaN      NaN        NaN   
Benito Juárez                     NaN        NaN      NaN        NaN   
San Ignacio Río Muerto            NaN        NaN      NaN        NaN   

                                                                               \
                                                                                
                            Asuntos Familiares             Estudio              
                                      Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                                    237       53.3       13        2.9   
Agua Prieta                                NaN        NaN      NaN        NaN   
Alamos                                     NaN        NaN      NaN        NaN   
Altar                                      NaN        NaN      NaN        NaN   
Arivechi                                   NaN        NaN      NaN        NaN   
...                                        ...        ...      ...        ...   
Villa Pesqueira                            NaN        NaN      NaN        NaN   
Yécora                                     NaN        NaN      NaN        NaN   
Gral. Plutarco Elías Calles                NaN        NaN      NaN        NaN   
Benito Juárez                              NaN        NaN      NaN        NaN   
San Ignacio Río Muerto                     NaN        NaN      NaN        NaN   

                                                                          \
                                                                           
                            Inseguridad delictiva o Violencia              
                                                     Absoluto Porcentaje   
Aconchi                                                     0          0   
Agua Prieta                                               NaN        NaN   
Alamos                                                    NaN        NaN   
Altar                                                     NaN        NaN   
Arivechi                                                  NaN        NaN   
...                                                       ...        ...   
Villa Pesqueira                                           NaN        NaN   
Yécora                                                    NaN        NaN   
Gral. Plutarco Elías Calles                               NaN        NaN   
Benito Juárez                                             NaN        NaN   
San Ignacio Río Muerto                                    NaN        NaN   

                                                 
                                                 
                                Otro             
                            Absoluto Porcentaje  
Aconchi                           97       21.8  
Agua Prieta                      NaN        NaN  
Alamos                           NaN        NaN  
Altar                            NaN        NaN  
Arivechi                         NaN        NaN  
...                              ...        ...  
Villa Pesqueira                  NaN        NaN  
Yécora                           NaN    

Ahora, lo aplicamos a los Municipios. 

In [306]:
for m,mun in enumerate(Municipios):
    df2 = sonora[mun][4].copy()
    df2.set_index('Unnamed: 1',inplace=True)
    df2 = df2.drop(index = 'nan', axis = 1)
    df2 = df2.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all')
    df2.columns = df2.iloc[0, :]
    df2 = df2.drop(index = ['Dinámica poblacional (2020)'], axis = 0)
    index = list(df2.index)
    extra_poblation = []  # Información Aidicional
    for i in index:
        if re.findall(feet_pattern, i):
            if m == 0:
                extra_poblation.append(i)
            df2 = df2.drop(index = i)
        if re.findall(inegi, i):
            df2 = df2.drop(index = i)
    if m == 0:
        extra_poblation[0] += 'y la otra mitad por encima.'
        extra_poblation[1] += '(15 a 64 años)'
    pob2 = df2.iloc[:-2,:2]
    migration = pob2.loc['Población de 5 años y más migrante':]
    migration_total = migration.iloc[0]
    migration_causes = migration.iloc[2:,:]
    migra_total = migration_total.tolist()
    migra_causes = dataframe_tolist(migration_causes)
    df_migra.iloc[m,:2] = migra_total
    df_migra.iloc[m,2:] = migra_causes
    

Finalmente, juntamtos todos los dataframes. 

In [307]:
dinamyc_poblation = pd.concat([df_pob,df_tcp,df_ind,df_migra],axis=1)



save_df(dinamyc_poblation, "DinamicaPoblacional2020.xlsx")

### Estructura por sexo. (5)

In [308]:
df = sonora[mun][5].copy()
df = df.set_index(df.columns[0])
extra_gender = []
note_pattern = "Nota"
for i in list(df.index):
    if re.findall(note_pattern, i):
        extra_gender.append(i)
        df = df.drop(index = i)
    if re.findall(inegi, i):
        df = df.drop(index = i)
df = df.replace({"nan":pd.NA}).dropna(axis = 1, how = 'all')
df.columns = df.iloc[1]
df = df.drop(index = ["Estructura de la población por edad y sexo, 2020"]).iloc[1:,:3]
df = df.set_index('Edad')
df

nan,% H,% M
Edad,,
85 y más,-0.392184326633518,0.371174451992436
80 a 84,-0.546256740668114,0.525246866027033
75 a 79,-0.875411443378388,0.658309405420548
70 a 74,-1.39365501785839,1.19055956299461
65 a 69,-1.68779326283353,1.6527768050984
60 a 64,-2.47916520764759,2.21304012886056
55 a 59,-2.41613558372435,2.45115204145949
50 a 54,-2.96239232439246,2.9203725751103
45 a 49,-3.22151411163247,3.12346802997409


#### Tabla Base

In [309]:
c0 = list(df.index)
c1 = ["%H", "%M"]

genderage_one = [one for j in range(len(c0)) for one in c1 ]
genderage_zero = [zero for zero in c0 for j in range(len(c1))]

mi_genderage = pd.MultiIndex.from_arrays([genderage_zero, genderage_one])
df_genage = pd.DataFrame(index = Municipios, columns= mi_genderage)


df_genage

85 y más      80 a 84      75 a 79      70 a 74  \
                                  %H   %M      %H   %M      %H   %M      %H   
Aconchi                          NaN  NaN     NaN  NaN     NaN  NaN     NaN   
Agua Prieta                      NaN  NaN     NaN  NaN     NaN  NaN     NaN   
Alamos                           NaN  NaN     NaN  NaN     NaN  NaN     NaN   
Altar                            NaN  NaN     NaN  NaN     NaN  NaN     NaN   
Arivechi                         NaN  NaN     NaN  NaN     NaN  NaN     NaN   
...                              ...  ...     ...  ...     ...  ...     ...   
Villa Pesqueira                  NaN  NaN     NaN  NaN     NaN  NaN     NaN   
Yécora                           NaN  NaN     NaN  NaN     NaN  NaN     NaN   
Gral. Plutarco Elías Calles      NaN  NaN     NaN  NaN     NaN  NaN     NaN   
Benito Juárez                    NaN  NaN     NaN  NaN     NaN  NaN     NaN   
San Ignacio Río Muerto           NaN  NaN     NaN  NaN     NaN  NaN     NaN   

                                 65 a 69       ... 20 a 24      15 a 19       \
                              %M      %H   %M  ...      %H   %M      %H   %M   
Aconchi                      NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
Agua Prieta                  NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
Alamos                       NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
Altar                        NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
Arivechi                     NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
...                          ...     ...  ...  ...     ...  ...     ...  ...   
Villa Pesqueira              NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
Yécora                       NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
Gral. Plutarco Elías Calles  NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
Benito Juárez                NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   
San Ignacio Río Muerto       NaN     NaN  NaN  ...     NaN  NaN     NaN  NaN   

                            10 a 14      5 a 9      0 a 4       
                                 %H   %M    %H   %M    %H   %M  
Aconchi                         NaN  NaN   NaN  NaN   NaN  NaN  
Agua Prieta                     NaN  NaN   NaN  NaN   NaN  NaN  
Alamos                          NaN  NaN   NaN  NaN   NaN  NaN  
Altar                           NaN  NaN   NaN  NaN   NaN  NaN  
Arivechi                        NaN  NaN   NaN  NaN   NaN  NaN  
...                             ...  ...   ...  ...   ...  ...  
Villa Pesqueira                 NaN  NaN   NaN  NaN   NaN  NaN  
Yécora                          NaN  NaN   NaN  NaN   NaN  NaN  
Gral. Plutarco Elías Calles     NaN  NaN   NaN  NaN   NaN  NaN  
Benito Juárez                   NaN  NaN   NaN  NaN   NaN  NaN  
San Ignacio Río Muerto          NaN  NaN   NaN  NaN   NaN  NaN  

[72 rows x 36 columns]

In [310]:
for m,mun in enumerate(Municipios):
    df = sonora[mun][5].copy()
    df = df.set_index(df.columns[0])
    extra_gender = []
    note_pattern = "Nota"
    for i in list(df.index):
        if re.findall(note_pattern, i):
            extra_gender.append(i)
            df = df.drop(index = i)
        if re.findall(inegi, i):
            df = df.drop(index = i)
    df = df.replace({"nan":pd.NA}).dropna(axis = 1, how = 'all')
    df.columns = df.iloc[1]
    df = df.drop(index = ["Estructura de la población por edad y sexo, 2020"]).iloc[1:,:3]
    df = df.set_index('Edad')
    gage = dataframe_tolist(df)
    df_genage.iloc[m] = gage


Finalmente hacemos el documento excel. 

In [311]:
save_df(df_genage, "EstructuraSexo2020.xlsx")


### Grupos Vulnerables (6)

In [312]:
df = sonora[mun][6].copy()
df.columns = df.iloc[0]
df = df.set_index('Grupos vulnerables (2020)')
df = df.replace({'nan':pd.NA}).dropna(axis = 1, how = 'all')
df = df.iloc[1:, :2]
df

116,Absoluto,Porcentaje
Grupos vulnerables (2020),,
Población de 3 años y más hablante de lengua indígena,931,6.9
Población de 3 años y más hablante de lengua indígena \nque no habla español,5,0.5
Población con discapacidad,916,6.4
Población con alguna limitación en la actividad,1846,12.9
Población con algún problema o condición mental,193,1.4
Personas en situación de pobreza 1,9675,57.2
Personas en pobreza extrema 1,2202,13
Personas en pobreza moderada 1,7473,44.2
Madres menores de 20 años 2,61,28.6


Ya vemos que algunas cosas han persistido con respecto al tiempo. Entonces vamos a tratar de convertirlas a funciones.

In [313]:
def footnote(data: pd.DataFrame):
    extra_information = []
    extra_codes = ['Nota','^\d']
    remove_codes = ['Fuente']
    for i in list(data.index):
        for code in extra_codes:
            if re.findall(code, i):
                extra_information.append(i)
                data = data.drop(index = i)
        for code in remove_codes:
            if re.findall(code, i):
                data = data.drop(index = i)
    return data, extra_information

df, extra_vul = footnote(df)
extra_vul

['1 El porcentaje se calculó respecto a la población total de 2020 estimada por CONEVAL.',
 '2 Información referida a 2022. Los datos se presentan por residencia habitual de la madre.']

#### Construcción de la Tabla

In [314]:
c0_vul = [ "Población","Población","Población",
          "Población","Población", "Población",
          "Población","Población","Población",
          "Población","Población", "Población",
          'Personas', 'Personas', 'Personas', 'Personas',
          'Personas', 'Personas', 'Personas', 'Personas']
    
    
c1_vul = ["Hablante de lengua indígena", "Hablante de lengua indígena", "Hablante de lengua indígena", "Hablante de lengua indígena",
          "Con discapacidad",  "Con discapacidad", "Con alguna limitación en la actividad", 
          "Con alguna limitación en la actividad", "Con algún problema o condición mental",
          "Con algún problema o condición mental", "Sin acta de nacimiento", "Sin acta de nacimiento",
          "Pobreza", "Pobreza", "Pobreza", "Pobreza", "Pobreza", "Pobreza", 
            "Madres menores de 20 años", "Madres menores de 20 años"]

c2_vul = ["De 3 años y más", "De 3 años y más", "De 3 años y más", "De 3 años y más",
          "","","", "", "","","", "", "(1)", "(1)", "(1)", "(1)", "(1)", "(1)", "(2)", "(2)"]

c3_vul = ['Total', 'Total', 'No habla español', 'No habla español',  "",
          "", "", "",  "", "",
          "", "", "Total", "Total", "Extrema",
          "Extrema", "Moderada", "Moderada", "", ""]
c4_vul = [last for j in range(len(c3_vul) // 2) for last in clast ]

mi_vul = pd.MultiIndex.from_arrays([c0_vul,c1_vul,c2_vul,c3_vul,c4_vul])

df_vul = pd.DataFrame(index = Municipios, columns=mi_vul)
df_vul.head()

Población                              \
            Hablante de lengua indígena                               
                        De 3 años y más                               
                                  Total            No habla español   
                               Absoluto Porcentaje         Absoluto   
Aconchi                             NaN        NaN              NaN   
Agua Prieta                         NaN        NaN              NaN   
Alamos                              NaN        NaN              NaN   
Altar                               NaN        NaN              NaN   
Arivechi                            NaN        NaN              NaN   

                                                    \
                       Con discapacidad              
                                                     
                                                     
            Porcentaje         Absoluto Porcentaje   
Aconchi            NaN              NaN        NaN   
Agua Prieta        NaN              NaN        NaN   
Alamos             NaN              NaN        NaN   
Altar              NaN              NaN        NaN   
Arivechi           NaN              NaN        NaN   

                                                              \
            Con alguna limitación en la actividad              
                                                               
                                                               
                                         Absoluto Porcentaje   
Aconchi                                       NaN        NaN   
Agua Prieta                                   NaN        NaN   
Alamos                                        NaN        NaN   
Altar                                         NaN        NaN   
Arivechi                                      NaN        NaN   

                                                              \
            Con algún problema o condición mental              
                                                               
                                                               
                                         Absoluto Porcentaje   
Aconchi                                       NaN        NaN   
Agua Prieta                                   NaN        NaN   
Alamos                                        NaN        NaN   
Altar                                         NaN        NaN   
Arivechi                                      NaN        NaN   

                                              Personas                      \
            Sin acta de nacimiento             Pobreza                       
                                                   (1)                       
                                                 Total             Extrema   
                          Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                        NaN        NaN      NaN        NaN      NaN   
Agua Prieta                    NaN        NaN      NaN        NaN      NaN   
Alamos                         NaN        NaN      NaN        NaN      NaN   
Altar                          NaN        NaN      NaN        NaN      NaN   
Arivechi                       NaN        NaN      NaN        NaN      NaN   

                                                                      \
                                           Madres menores de 20 años   
                                                                 (2)   
                       Moderada                                        
            Porcentaje Absoluto Porcentaje                  Absoluto   
Aconchi            NaN      NaN        NaN                       NaN   
Agua Prieta        NaN      NaN        NaN                       NaN   
Alamos             NaN      NaN        NaN                       NaN   
Altar              NaN      NaN        NaN                       NaN   
Arivechi           NaN      NaN        NaN                       Na

Ahora, rellenaremos la tabla.

In [315]:
snacta = df.iloc[-1]
pob_vul = df.iloc[:5,:]
per_vul = df.iloc[5:-1,]

poblation = dataframe_tolist(pob_vul)
people = dataframe_tolist(per_vul)

df_vul.iloc[0, :len(poblation)] = poblation
df_vul.iloc[0,len(poblation): len(poblation) + 2] = snacta
df_vul.iloc[0, len(poblation) + 2: ] = people
df_vul.head()

Población                              \
            Hablante de lengua indígena                               
                        De 3 años y más                               
                                  Total            No habla español   
                               Absoluto Porcentaje         Absoluto   
Aconchi                             931        6.9                5   
Agua Prieta                         NaN        NaN              NaN   
Alamos                              NaN        NaN              NaN   
Altar                               NaN        NaN              NaN   
Arivechi                            NaN        NaN              NaN   

                                                    \
                       Con discapacidad              
                                                     
                                                     
            Porcentaje         Absoluto Porcentaje   
Aconchi            0.5              916        6.4   
Agua Prieta        NaN              NaN        NaN   
Alamos             NaN              NaN        NaN   
Altar              NaN              NaN        NaN   
Arivechi           NaN              NaN        NaN   

                                                              \
            Con alguna limitación en la actividad              
                                                               
                                                               
                                         Absoluto Porcentaje   
Aconchi                                      1846       12.9   
Agua Prieta                                   NaN        NaN   
Alamos                                        NaN        NaN   
Altar                                         NaN        NaN   
Arivechi                                      NaN        NaN   

                                                              \
            Con algún problema o condición mental              
                                                               
                                                               
                                         Absoluto Porcentaje   
Aconchi                                       193        1.4   
Agua Prieta                                   NaN        NaN   
Alamos                                        NaN        NaN   
Altar                                         NaN        NaN   
Arivechi                                      NaN        NaN   

                                              Personas                      \
            Sin acta de nacimiento             Pobreza                       
                                                   (1)                       
                                                 Total             Extrema   
                          Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi                        151        1.1     9675       57.2     2202   
Agua Prieta                    NaN        NaN      NaN        NaN      NaN   
Alamos                         NaN        NaN      NaN        NaN      NaN   
Altar                          NaN        NaN      NaN        NaN      NaN   
Arivechi                       NaN        NaN      NaN        NaN      NaN   

                                                                      \
                                           Madres menores de 20 años   
                                                                 (2)   
                       Moderada                                        
            Porcentaje Absoluto Porcentaje                  Absoluto   
Aconchi             13     7473       44.2                        61   
Agua Prieta        NaN      NaN        NaN                       NaN   
Alamos             NaN      NaN        NaN                       NaN   
Altar              NaN      NaN        NaN                       NaN   
Arivechi           NaN      NaN        NaN                       Na

Ahora, con todos los municipios. 

In [316]:
for m, mun in enumerate(Municipios):
    df = sonora[mun][6].copy()
    df.columns = df.iloc[0]
    df = df.set_index('Grupos vulnerables (2020)')
    df = df.replace({'nan':pd.NA}).dropna(axis = 1, how = 'all')
    df = df.iloc[1:, :2]
    df, extra_vul = footnote(df)
    snacta = df.iloc[-1]
    pob_vul = df.iloc[:5,:]
    per_vul = df.iloc[5:-1,]

    poblation = dataframe_tolist(pob_vul)
    people = dataframe_tolist(per_vul)
    df_vul.iloc[m, :len(poblation)] = poblation
    df_vul.iloc[m,len(poblation): len(poblation) + 2] = snacta
    df_vul.iloc[m, len(poblation) + 2: ] = people

In [317]:
vuln = df_vul.astype(float)
save_df(vuln, "GruposVulnerables2020.xlsx")


### Hechos Vitales (7)

In [318]:
df = sonora[mun][7].copy()
df.columns = df.iloc[0]
df.drop(index = df.index[0],inplace=True)
df.set_index(df.columns[0], inplace= True)
df, extra_vitals = footnote(df)
df = df.replace({"nan" : pd.NA}).dropna(axis = 1, how = "all").iloc[:, :2]

def format_df(df: pd.DataFrame):
    df.columns = df.iloc[0]
    df.drop(index = df.index[0],inplace=True)
    df.set_index(df.columns[0], inplace= True)
    df, extra = footnote(df)
    df = df.replace({"nan" : pd.NA}).dropna(axis = 1, how = "all").iloc[:, :2]
    return df, extra

df

136,Absoluto,Porcentaje
Hechos vitales (2022),,
Nacimientos 1,213,-
Defunciones totales 2,125,-
Defunciones por accidente,7,5.6
Defunciones por homicidio,8,6.4
Defunciones por suicidio,1,0.8
Defunciones fetales,1,-
Matrimonios,56,-
Divorcios,0,-


#### Creacion de la Tabla

In [319]:
vitals_c0 = ['Nacimientos', 'Nacimientos',
              'Defunciones', 'Defunciones', 'Defunciones', 'Defunciones', 'Defunciones',
              'Defunciones', 'Defunciones', 'Defunciones', 'Defunciones', 'Defunciones',
              'Matrimonios', 'Matrimonios', 'Divorcios', 'Divorcios']
vitals_c1 = ["", "", "Total","Total", "Causa","Causa", "Causa", "Causa","Causa", "Causa", 
             "Fetales", "Fetales",
             "","", "",""]
vitals_c2 = ["","", "","", "Accidente", "Accidente", "Homicidio", "Homicidio",
            "Suicidio", "Suicidio","",
             "","", "","", ""]
vitals_c3 = [three for j in range(len(vitals_c2) // 2) for three in clast]

mi_vitals = pd.MultiIndex.from_arrays([vitals_c0, vitals_c1, vitals_c2, vitals_c3])
vitals_df = pd.DataFrame(index = Municipios, columns = mi_vitals)


vitals_df.head()

Nacimientos            Defunciones                       \
                                         Total                Causa   
                                                          Accidente   
               Absoluto Porcentaje    Absoluto Porcentaje  Absoluto   
Aconchi             NaN        NaN         NaN        NaN       NaN   
Agua Prieta         NaN        NaN         NaN        NaN       NaN   
Alamos              NaN        NaN         NaN        NaN       NaN   
Altar               NaN        NaN         NaN        NaN       NaN   
Arivechi            NaN        NaN         NaN        NaN       NaN   

                                                                          \
                                                                 Fetales   
                       Homicidio            Suicidio                       
            Porcentaje  Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi            NaN       NaN        NaN      NaN        NaN      NaN   
Agua Prieta        NaN       NaN        NaN      NaN        NaN      NaN   
Alamos             NaN       NaN        NaN      NaN        NaN      NaN   
Altar              NaN       NaN        NaN      NaN        NaN      NaN   
Arivechi           NaN       NaN        NaN      NaN        NaN      NaN   

                       Matrimonios            Divorcios             
                                                                    
                                                                    
            Porcentaje    Absoluto Porcentaje  Absoluto Porcentaje  
Aconchi            NaN         NaN        NaN       NaN        NaN  
Agua Prieta        NaN         NaN        NaN       NaN        NaN  
Alamos             NaN         NaN        NaN       NaN        NaN  
Altar              NaN         NaN        NaN       NaN        NaN  
Arivechi           NaN         NaN        NaN       NaN        NaN

In [320]:
vitals_data = dataframe_tolist(df)
vitals_df.iloc[0] = vitals_data
vitals_df.head()

Nacimientos            Defunciones                       \
                                         Total                Causa   
                                                          Accidente   
               Absoluto Porcentaje    Absoluto Porcentaje  Absoluto   
Aconchi             213          -         125          -         7   
Agua Prieta         NaN        NaN         NaN        NaN       NaN   
Alamos              NaN        NaN         NaN        NaN       NaN   
Altar               NaN        NaN         NaN        NaN       NaN   
Arivechi            NaN        NaN         NaN        NaN       NaN   

                                                                          \
                                                                 Fetales   
                       Homicidio            Suicidio                       
            Porcentaje  Absoluto Porcentaje Absoluto Porcentaje Absoluto   
Aconchi            5.6         8        6.4        1        0.8        1   
Agua Prieta        NaN       NaN        NaN      NaN        NaN      NaN   
Alamos             NaN       NaN        NaN      NaN        NaN      NaN   
Altar              NaN       NaN        NaN      NaN        NaN      NaN   
Arivechi           NaN       NaN        NaN      NaN        NaN      NaN   

                       Matrimonios            Divorcios             
                                                                    
                                                                    
            Porcentaje    Absoluto Porcentaje  Absoluto Porcentaje  
Aconchi              -          56          -         0          -  
Agua Prieta        NaN         NaN        NaN       NaN        NaN  
Alamos             NaN         NaN        NaN       NaN        NaN  
Altar              NaN         NaN        NaN       NaN        NaN  
Arivechi           NaN         NaN        NaN       NaN        NaN

Ahora, lo aplicamos a todos los municipios. 

In [321]:
for m, mun in enumerate(Municipios):
    df = sonora[mun][7].copy()
    df, extra_vitals = format_df(df)
    vitals_data = dataframe_tolist(df)
    vitals_df.iloc[m] = vitals_data

save_df(vitals_df, "HechosVitales2020.xlsx")

### Salud (8/9)

In [322]:
df = sonora[mun][8].copy()
df, extra_health = format_df(df)

df2 = sonora[mun][9].copy()
extra_health.append(df2.iloc[0,0])
extra_health.append(df2.iloc[1,0])

df

150,Absoluto,Porcentaje
Salud (2022),,
Población afiliada a servicios de salud 1,11118,77.9
Población usuaria de servicios de salud 1,13505,94.8
Tasa de mortalidad 2,-,7.8
Tasa de mortalidad infantil 3,-,9.4
Unidades médicas 4,3,-
Unidades de consulta externa,3,100
Unidades de hospitalización general,0,0
Unidades de hospitalización especializada,0,0


#### Construcción de la Tabla

In [323]:
health_c0 = ["Servicios de Salud (1)", "Servicios de Salud (1)", "Servicios de Salud (1)", "Servicios de Salud (1)",
             "Tasa de mortalidad", "Tasa de mortalidad", "Tasa de mortalidad infantil", "Tasa de mortalidad infantil",
             "Unidades médicas", "Unidades médicas", "Unidades de consulta externa", "Unidades de consulta externa",
             "Unidades de hospitalización", "Unidades de hospitalización", "Unidades de hospitalización", "Unidades de hospitalización"]



health_c1 = ["Población Afiliada", "Población Afiliada", "Poblacion Usuaria", "Poblacion Usuaria",
             "","","","",
             "", "", "", "",
             "General", "General", "Especializada", "Especializada"]

health_c2 = [last for j in range(len(health_c1) // 2 ) for last in clast]

mi_health = pd.MultiIndex.from_arrays([health_c0, health_c1, health_c2])
df_health = pd.DataFrame(index = Municipios, columns= mi_health)
df_health.head()

Servicios de Salud (1)                                          \
                Población Afiliada            Poblacion Usuaria              
                          Absoluto Porcentaje          Absoluto Porcentaje   
Aconchi                        NaN        NaN               NaN        NaN   
Agua Prieta                    NaN        NaN               NaN        NaN   
Alamos                         NaN        NaN               NaN        NaN   
Altar                          NaN        NaN               NaN        NaN   
Arivechi                       NaN        NaN               NaN        NaN   

            Tasa de mortalidad            Tasa de mortalidad infantil  \
                                                                        
                      Absoluto Porcentaje                    Absoluto   
Aconchi                    NaN        NaN                         NaN   
Agua Prieta                NaN        NaN                         NaN   
Alamos                     NaN        NaN                         NaN   
Altar                      NaN        NaN                         NaN   
Arivechi                   NaN        NaN                         NaN   

                       Unidades médicas             \
                                                     
            Porcentaje         Absoluto Porcentaje   
Aconchi            NaN              NaN        NaN   
Agua Prieta        NaN              NaN        NaN   
Alamos             NaN              NaN        NaN   
Altar              NaN              NaN        NaN   
Arivechi           NaN              NaN        NaN   

            Unidades de consulta externa             \
                                                      
                                Absoluto Porcentaje   
Aconchi                              NaN        NaN   
Agua Prieta                          NaN        NaN   
Alamos                               NaN        NaN   
Altar                                NaN        NaN   
Arivechi                             NaN        NaN   

            Unidades de hospitalización                                      
                                General            Especializada             
                               Absoluto Porcentaje      Absoluto Porcentaje  
Aconchi                             NaN        NaN           NaN        NaN  
Agua Prieta                         NaN        NaN           NaN        NaN  
Alamos                              NaN        NaN           NaN        NaN  
Altar                               NaN        NaN           NaN        NaN  
Arivechi                            NaN        NaN           NaN        NaN

In [324]:
health_data = dataframe_tolist(df)

df_health.iloc[0] = health_data
df_health.head()

Servicios de Salud (1)                                          \
                Población Afiliada            Poblacion Usuaria              
                          Absoluto Porcentaje          Absoluto Porcentaje   
Aconchi                      11118       77.9             13505       94.8   
Agua Prieta                    NaN        NaN               NaN        NaN   
Alamos                         NaN        NaN               NaN        NaN   
Altar                          NaN        NaN               NaN        NaN   
Arivechi                       NaN        NaN               NaN        NaN   

            Tasa de mortalidad            Tasa de mortalidad infantil  \
                                                                        
                      Absoluto Porcentaje                    Absoluto   
Aconchi                      -        7.8                           -   
Agua Prieta                NaN        NaN                         NaN   
Alamos                     NaN        NaN                         NaN   
Altar                      NaN        NaN                         NaN   
Arivechi                   NaN        NaN                         NaN   

                       Unidades médicas             \
                                                     
            Porcentaje         Absoluto Porcentaje   
Aconchi            9.4                3          -   
Agua Prieta        NaN              NaN        NaN   
Alamos             NaN              NaN        NaN   
Altar              NaN              NaN        NaN   
Arivechi           NaN              NaN        NaN   

            Unidades de consulta externa             \
                                                      
                                Absoluto Porcentaje   
Aconchi                                3        100   
Agua Prieta                          NaN        NaN   
Alamos                               NaN        NaN   
Altar                                NaN        NaN   
Arivechi                             NaN        NaN   

            Unidades de hospitalización                                      
                                General            Especializada             
                               Absoluto Porcentaje      Absoluto Porcentaje  
Aconchi                               0          0             0          0  
Agua Prieta                         NaN        NaN           NaN        NaN  
Alamos                              NaN        NaN           NaN        NaN  
Altar                               NaN        NaN           NaN        NaN  
Arivechi                            NaN        NaN           NaN        NaN

In [325]:
for m, mun in enumerate(Municipios):
    df = sonora[mun][8].copy()
    df, extra_health = format_df(df)
    if m == 0:
        df2 = sonora[mun][9].copy()
        extra_health.append(df2.iloc[0,0])
        extra_health.append(df2.iloc[1,0])
    health_data = dataframe_tolist(df)
    df_health.iloc[m] = health_data

save_df(df_health, "Salud2022.xlsx")
df_health.head()


Servicios de Salud (1)                                          \
                Población Afiliada            Poblacion Usuaria              
                          Absoluto Porcentaje          Absoluto Porcentaje   
Aconchi                       2026         79              2518       98.2   
Agua Prieta                  65050       70.8             88416       96.6   
Alamos                       20992         84             24524       98.3   
Altar                         6409       67.5              9213       97.3   
Arivechi                       934       79.4              1125       95.6   

            Tasa de mortalidad            Tasa de mortalidad infantil  \
                                                                        
                      Absoluto Porcentaje                    Absoluto   
Aconchi                      -        6.4                           -   
Agua Prieta                  -        5.7                           -   
Alamos                       -        8.6                           -   
Altar                        -        7.7                           -   
Arivechi                     -        5.3                           -   

                       Unidades médicas             \
                                                     
            Porcentaje         Absoluto Porcentaje   
Aconchi              0                2          -   
Agua Prieta       10.3                8          -   
Alamos             5.4               21          -   
Altar                0                3          -   
Arivechi             0                2          -   

            Unidades de consulta externa             \
                                                      
                                Absoluto Porcentaje   
Aconchi                                2        100   
Agua Prieta                            6         75   
Alamos                                20       95.2   
Altar                                  3        100   
Arivechi                               2        100   

            Unidades de hospitalización                                      
                                General            Especializada             
                               Absoluto Porcentaje      Absoluto Porcentaje  
Aconchi                               0          0             0          0  
Agua Prieta                           2         25             0          0  
Alamos                                1        4.8             0          0  
Altar                                 0          0             0          0  
Arivechi                              0          0             0          0

### Educación (10/11)

In [326]:
df = sonora[mun][10].copy()
df, extra_edu = format_df(df)

df = df.drop(index = ['nan','Educación (2020)'])
df

166,Absoluto,Porcentaje
Educación (2020),,
Población analfabeta 1,449,4.3
Grado promedio de escolaridad 1,8.6,-
Población de 3 años y más que asiste a la escuela 2,3560,26.2
Población de 3 a 5 años,392,53.6
Población de 6 a 11 años,1433,95
Población de 12 a 14 años,722,90.8
Población de 15 a 17 años,510,65.6
Población de 18 a 24 años,436,25.7
Población de 25 años y más,67,0.8


En este caso, vamos a divir la tabla en 4 partes.

* Intro
* Educación por Edad
* Educación por Nivel
* Planteles.

Comenzaremos con la Intro, que es la parte más sencilla.

In [327]:
edu_intro = df.iloc[:2]
edu_intro

166,Absoluto,Porcentaje
Educación (2020),,
Población analfabeta 1,449,4.3
Grado promedio de escolaridad 1,8.6,-


In [328]:
edu_intro = df.iloc[:2]
edu_intro

edu_intro_c0 = ['Población Analfabeta','Población Analfabeta',
                'Grado Promedio de Escolaridad', 'Grado Promedio de Escolaridad']
edu_intro_c1 = [last for j in range(len(edu_intro_c0) // 2) for last in clast]

mi_edu_intro = pd.MultiIndex.from_arrays([edu_intro_c0, edu_intro_c1])
edu_df_intro = pd.DataFrame(index = Municipios, columns=mi_edu_intro)

for m,mun in enumerate(Municipios):
    df = sonora[mun][10].copy()
    df, extra_edu = format_df(df)
    df = df.drop(index = ['nan','Educación (2020)'])
    edu_intro = df.iloc[:2]
    intro_data = dataframe_tolist(edu_intro)
    edu_df_intro.iloc[m] = intro_data

edu_df_intro.head()

Población Analfabeta            Grado Promedio de Escolaridad  \
                        Absoluto Porcentaje                      Absoluto   
Aconchi                       53        2.7                           8.8   
Agua Prieta                  989        1.5                           9.6   
Alamos                      1508        8.1                           8.2   
Altar                        231        3.4                           8.7   
Arivechi                      29        3.2                           8.2   

                        
            Porcentaje  
Aconchi              -  
Agua Prieta          -  
Alamos               -  
Altar                -  
Arivechi             -

Ahora vamos con las partes más complejas. Primero debemos separarlas. Para ello usaremos regex.

In [329]:
edu_rest = df.iloc[2:]
edu_rest

166,Absoluto,Porcentaje
Educación (2020),,
Población de 3 años y más que asiste a la escuela 2,3560,26.2
Población de 3 a 5 años,392,53.6
Población de 6 a 11 años,1433,95
Población de 12 a 14 años,722,90.8
Población de 15 a 17 años,510,65.6
Población de 18 a 24 años,436,25.7
Población de 25 años y más,67,0.8
Población de 3 años y más según nivel de escolaridad 2,13563,-
Población sin escolaridad,883,6.5


In [330]:
age_pattern = "\d+\sa\s\d+| 25"
plantel_pattern = "Plantel"
escol_pattern = "Población (con|sin)"
edu_index = list(edu_rest.index)

age_index = [edu_rest.index[0]]
plantel_index = [edu_rest.index[12]]
escol_index = [edu_rest.index[7]]
for i in edu_index:
    if re.findall(age_pattern, i):
        age_index.append(i)
    if re.findall(plantel_pattern, i):
        plantel_index.append(i)
    if re.findall(escol_pattern, i):
        escol_index.append(i)


Ahora creamos los distintos dataframes.

In [331]:
edu_age = edu_rest.loc[age_index]
edu_plantel = edu_rest.loc[plantel_index]
edu_escol = edu_rest.loc[escol_index]

edu_escol

166,Absoluto,Porcentaje
Educación (2020),,
Población de 3 años y más según nivel de escolaridad 2,13563,-
Población sin escolaridad,883,6.5
Población con educación básica,9297,68.5
Población con educación media superior,2146,15.8
Población con educación superior,1220,9


Ahora trabajaremos con cada bloque por separado.

In [332]:
education = {
    "cat0": ['Educación','Planteles'],
    "cat1": ['Edad','Escolaridad'],
    "cat1": ['Planteles'],
    "age" : ["Total", "3-5","6-11","12-14","15-17","18-24","+25"],
    "plantel" : ["Total", "Educación Inicial","Preescolar","Primaria","Secundaria","Bachillerato"],
    "escol" : ["Total", "Sin Escolaridad","Básica","Media Superior","Superior"]
}

edu_age_c2 = [age for age in education["age"] for j in range(len(clast))]
edu_pl_c2 = [pl for pl in education["plantel"] for j in range(len(clast))]
edu_escol_c2 = [escol for escol in education["escol"] for j in range(len(clast))]

edu_escage_c1 = ['Edad' for j in range(len(edu_age_c2))] + ['Escolaridad' for j in range(len(edu_escol_c2))]
edu_pl_c1 = ['Sector' for j in range(len(edu_pl_c2))]

edu_c1 = edu_escage_c1 + edu_pl_c1
edu_c2 = edu_age_c2 + edu_escol_c2 + edu_pl_c2
edu_c0 = ['Educación' for j in range(len(edu_escage_c1))] + ['Plantel' for j in range(len(edu_pl_c1))]
edu_c3 = [last  for j in range(len(edu_c1) // 2) for last in clast]
mi_edu = pd.MultiIndex.from_arrays([edu_c0,edu_c1,edu_c2,edu_c3])
df_edu = pd.DataFrame(index = Municipios, columns= mi_edu)
df_edu.head()

Educación                                                     \
                 Edad                                                      
                Total                 3-5                6-11              
             Absoluto Porcentaje Absoluto Porcentaje Absoluto Porcentaje   
Aconchi           NaN        NaN      NaN        NaN      NaN        NaN   
Agua Prieta       NaN        NaN      NaN        NaN      NaN        NaN   
Alamos            NaN        NaN      NaN        NaN      NaN        NaN   
Altar             NaN        NaN      NaN        NaN      NaN        NaN   
Arivechi          NaN        NaN      NaN        NaN      NaN        NaN   

                                                     ...           Plantel  \
                                                     ...            Sector   
               12-14               15-17             ... Educación Inicial   
            Absoluto Porcentaje Absoluto Porcentaje  ...          Absoluto   
Aconchi          NaN        NaN      NaN        NaN  ...               NaN   
Agua Prieta      NaN        NaN      NaN        NaN  ...               NaN   
Alamos           NaN        NaN      NaN        NaN  ...               NaN   
Altar            NaN        NaN      NaN        NaN  ...               NaN   
Arivechi         NaN        NaN      NaN        NaN  ...               NaN   

                                                                             \
                                                                              
                       Preescolar            Primaria            Secundaria   
            Porcentaje   Absoluto Porcentaje Absoluto Porcentaje   Absoluto   
Aconchi            NaN        NaN        NaN      NaN        NaN        NaN   
Agua Prieta        NaN        NaN        NaN      NaN        NaN        NaN   
Alamos             NaN        NaN        NaN      NaN        NaN        NaN   
Altar              NaN        NaN        NaN      NaN        NaN        NaN   
Arivechi           NaN        NaN        NaN      NaN        NaN        NaN   

                                                
                                                
                       Bachillerato             
            Porcentaje     Absoluto Porcentaje  
Aconchi            NaN          NaN        NaN  
Agua Prieta        NaN          NaN        NaN  
Alamos             NaN          NaN        NaN  
Altar              NaN          NaN        NaN  
Arivechi           NaN          NaN        NaN  

[5 rows x 36 columns]

In [333]:
print(len(dataframe_tolist(edu_rest)))
df_edu.shape

36


(72, 36)

In [334]:


edu_intro_c0 = ['Población Analfabeta','Población Analfabeta',
                    'Grado Promedio de Escolaridad', 'Grado Promedio de Escolaridad']
edu_intro_c1 = [last for j in range(len(edu_intro_c0) // 2) for last in clast]
cm1 = ["" for j in range(len(edu_intro_c0))]
cm2 = cm1.copy()
mi_edu_intro = pd.MultiIndex.from_arrays([cm2, cm1, edu_intro_c0, edu_intro_c1])
edu_df_intro = pd.DataFrame(index = Municipios, columns=mi_edu_intro)
for m,mun in enumerate(Municipios):
    df = sonora[mun][10].copy()
    df, extra_edu = format_df(df)
    df = df.drop(index = ['nan','Educación (2020)'])
    edu_intro = df.iloc[:2]
    intro_data = dataframe_tolist(edu_intro)
    edu_df_intro.iloc[m] = intro_data

education = {
    "cat0": ['Educación','Planteles'],
    "cat1": ['Edad','Escolaridad'],
    "cat1": ['Planteles'],
    "age" : ["Total", "3-5","6-11","12-14","15-17","18-24","+25"],
    "plantel" : ["Total", "Educación Inicial","Preescolar","Primaria","Secundaria","Bachillerato"],
    "escol" : ["Total", "Sin Escolaridad","Básica","Media Superior","Superior"]
}

edu_age_c2 = [age for age in education["age"] for j in range(len(clast))]
edu_pl_c2 = [pl for pl in education["plantel"] for j in range(len(clast))]
edu_escol_c2 = [escol for escol in education["escol"] for j in range(len(clast))]

edu_escage_c1 = ['Edad' for j in range(len(edu_age_c2))] + ['Escolaridad' for j in range(len(edu_escol_c2))]
edu_pl_c1 = ['Sector' for j in range(len(edu_pl_c2))]

edu_c1 = edu_escage_c1 + edu_pl_c1
edu_c2 = edu_age_c2 + edu_escol_c2 + edu_pl_c2
edu_c0 = ['Educación' for j in range(len(edu_escage_c1))] + ['Plantel' for j in range(len(edu_pl_c1))]
edu_c3 = [last  for j in range(len(edu_c1) // 2) for last in clast]
mi_edu = pd.MultiIndex.from_arrays([edu_c0,edu_c1,edu_c2,edu_c3])
df_edu = pd.DataFrame(index = Municipios, columns= mi_edu)
df_edu.head()

for m, mun in enumerate(Municipios):
    # df Resto, Nuevo Formato
    df = sonora[mun][10].copy()
    df, extra_edu = format_df(df)
    df = df.drop(index = ['nan','Educación (2020)'])
    edu_rest = df.iloc[2:]
    rest_data = dataframe_tolist(edu_rest)
    df_edu.iloc[m] = rest_data
df_edu = pd.concat([edu_df_intro, df_edu], axis = 1)


save_df(df_edu, "Educacion2020.xlsx")


### Hoggares y Vivienda 2020 (12)

In [335]:
df = sonora[mun][12].copy()
df, extra_home = format_df(df)
df

237,Absoluto,Porcentaje
Hogares y viviendas (2020),,
Tenencia de la vivienda,<NA>,<NA>
Viviendas propias,3280,83
Viviendas alquiladas,148,3.7
Viviendas de un familiar o prestadas,505,12.8
Disponibilidad de escrituras 4,2138,65.2


In [336]:
c0 = list(df.index)
home_one = [last for j in range(len(c0)) for last in clast]
home_zero = [one for one in c0 for j in range(len(clast))]

mi_home = pd.MultiIndex.from_arrays([home_zero, home_one])

home2020 = pd.DataFrame(index = Municipios, columns= mi_home)
home2020.head()


Tenencia de la vivienda            Viviendas propias             \
                           Absoluto Porcentaje          Absoluto Porcentaje   
Aconchi                         NaN        NaN               NaN        NaN   
Agua Prieta                     NaN        NaN               NaN        NaN   
Alamos                          NaN        NaN               NaN        NaN   
Altar                           NaN        NaN               NaN        NaN   
Arivechi                        NaN        NaN               NaN        NaN   

            Viviendas alquiladas             \
                        Absoluto Porcentaje   
Aconchi                      NaN        NaN   
Agua Prieta                  NaN        NaN   
Alamos                       NaN        NaN   
Altar                        NaN        NaN   
Arivechi                     NaN        NaN   

            Viviendas de un familiar o prestadas             \
                                        Absoluto Porcentaje   
Aconchi                                      NaN        NaN   
Agua Prieta                                  NaN        NaN   
Alamos                                       NaN        NaN   
Altar                                        NaN        NaN   
Arivechi                                     NaN        NaN   

            Disponibilidad de escrituras 4             
                                  Absoluto Porcentaje  
Aconchi                                NaN        NaN  
Agua Prieta                            NaN        NaN  
Alamos                                 NaN        NaN  
Altar                                  NaN        NaN  
Arivechi                               NaN        NaN

In [337]:
for m, mun in enumerate(Municipios):
    df = sonora[mun][12].copy()
    df, extra_home = format_df(df)
    home_data = dataframe_tolist(df)
    home2020.iloc[m] = home_data

save_df(home2020, "HogaresViviendas2020.xlsx")


### Medio Ambiente 2020 (13)

In [338]:
df = sonora[mun][13].copy()
df, extra_eco = format_df(df)

df

250,Absoluto,Porcentaje
Medio ambiente (2020),,
Total de puntos de descarga de aguas residuales \nmunicipales sin tratamiento,1,-
Descarga a río o arroyo,0,0
Descarga a lago o laguna,0,0
Descarga a suelo o barranca,0,0
Descarga a canal o dren,0,0
Otro punto,1,100


In [339]:
c0 = list(df.index)
c0[0] = c0[0].replace('\n', ' ')
eco_one = [last for j in range(len(c0)) for last in clast]
eco_zero = [one for one in c0 for j in range(len(clast))]

mi_eco = pd.MultiIndex.from_arrays([eco_zero, eco_one])

eco2020 = pd.DataFrame(index = Municipios, columns= mi_eco)
eco2020.head()

Total de puntos de descarga de aguas residuales  municipales sin tratamiento  \
                                                                                Absoluto   
Aconchi                                                    NaN                             
Agua Prieta                                                NaN                             
Alamos                                                     NaN                             
Altar                                                      NaN                             
Arivechi                                                   NaN                             

                       Descarga a río o arroyo             \
            Porcentaje                Absoluto Porcentaje   
Aconchi            NaN                     NaN        NaN   
Agua Prieta        NaN                     NaN        NaN   
Alamos             NaN                     NaN        NaN   
Altar              NaN                     NaN        NaN   
Arivechi           NaN                     NaN        NaN   

            Descarga a lago o laguna            Descarga a suelo o barranca  \
                            Absoluto Porcentaje                    Absoluto   
Aconchi                          NaN        NaN                         NaN   
Agua Prieta                      NaN        NaN                         NaN   
Alamos                           NaN        NaN                         NaN   
Altar                            NaN        NaN                         NaN   
Arivechi                         NaN        NaN                         NaN   

                       Descarga a canal o dren            Otro punto  \
            Porcentaje                Absoluto Porcentaje   Absoluto   
Aconchi            NaN                     NaN        NaN        NaN   
Agua Prieta        NaN                     NaN        NaN        NaN   
Alamos             NaN                     NaN        NaN        NaN   
Altar              NaN                     NaN        NaN        NaN   
Arivechi           NaN                     NaN        NaN        NaN   

                        
            Porcentaje  
Aconchi            NaN  
Agua Prieta        NaN  
Alamos             NaN  
Altar              NaN  
Arivechi           NaN

In [340]:
for m, mun in enumerate(Municipios):
    df = sonora[mun][13].copy()
    df, extra_eco = format_df(df)
    eco_data = dataframe_tolist(df)
    eco2020.iloc[m] = eco_data


save_df(eco2020, "MedioAmbiente2020.xlsx")


### Entorno Urbano 2020 (14)

In [341]:
df = sonora[mun][14].copy()
df, extra_urban = format_df(df)
df

260,Absoluto,Porcentaje
Entorno urbano (2020),,
Total de manzanas en localidades de 2 500 habitantes y más,300,-
Alumbrado público en todas las vialidades,18,6
Alumbrado público en alguna vialidad,218,72.7
Alumbrado público en ninguna vialidad,64,21.3
Recubrimiento en todas las vialidades,35,11.7
Recubrimiento en alguna vialidad,186,62
Recubrimiento en ninguna vialidad,79,26.3
Guarnición en todas las vialidades,41,13.7
Guarnición en alguna vialidad,170,56.7


##### Construcción de la Tabla

In [342]:
c0_1 = df.index[0]
c0_2 = ['Alumbrado Público', 'Recubrimiento', 'Guarnición', 'Rampa para silla de Ruedas']
c1 = ['Todas', 'Alguna', 'Ninguna']

u0 = [c0 for c0 in c0_2 for j in range(len(clast) * len(c1))]
u1 = len(c0_2) * ['Todas', 'Todas', 'Alguna', 'Alguna', 'Ninguna', 'Ninguna']
u2 = [c  for j in range(len(c1) * len(c0_2)) for c in clast]

v0 = [c0_1, c0_1]
v1 = ["",""]
v2 = clast

urban_zero = v0 + u0
urban_one = v1 + u1
urban_two = v2 + u2

mi = pd.MultiIndex.from_arrays([urban_zero, urban_one, urban_two])
urban2020 = pd.DataFrame(index = Municipios, columns=mi)
urban2020


Total de manzanas en localidades de 2 500 habitantes y más  \
                                                                                         
                                                                              Absoluto   
Aconchi                                                                    NaN           
Agua Prieta                                                                NaN           
Alamos                                                                     NaN           
Altar                                                                      NaN           
Arivechi                                                                   NaN           
...                                                                        ...           
Villa Pesqueira                                                            NaN           
Yécora                                                                     NaN           
Gral. Plutarco Elías Calles                                                NaN           
Benito Juárez                                                              NaN           
San Ignacio Río Muerto                                                     NaN           

                                       Alumbrado Público                      \
                                                   Todas              Alguna   
                            Porcentaje          Absoluto Porcentaje Absoluto   
Aconchi                            NaN               NaN        NaN      NaN   
Agua Prieta                        NaN               NaN        NaN      NaN   
Alamos                             NaN               NaN        NaN      NaN   
Altar                              NaN               NaN        NaN      NaN   
Arivechi                           NaN               NaN        NaN      NaN   
...                                ...               ...        ...      ...   
Villa Pesqueira                    NaN               NaN        NaN      NaN   
Yécora                             NaN               NaN        NaN      NaN   
Gral. Plutarco Elías Calles        NaN               NaN        NaN      NaN   
Benito Juárez                      NaN               NaN        NaN      NaN   
San Ignacio Río Muerto             NaN               NaN        NaN      NaN   

                                                           Recubrimiento  \
                                        Ninguna                    Todas   
                            Porcentaje Absoluto Porcentaje      Absoluto   
Aconchi                            NaN      NaN        NaN           NaN   
Agua Prieta                        NaN      NaN        NaN           NaN   
Alamos                             NaN      NaN        NaN           NaN   
Altar                              NaN      NaN        NaN           NaN   
Arivechi                           NaN      NaN        NaN           NaN   
...                                ...      ...        ...           ...   
Villa Pesqueira                    NaN      NaN        NaN           NaN   
Yécora                             NaN      NaN        NaN           NaN   
Gral. Plutarco Elías Calles        NaN      NaN        NaN           NaN   
Benito Juárez                      NaN      NaN        NaN           NaN   
San Ignacio Río Muerto             NaN      NaN        NaN           NaN   

                                        ... Guarnición                      \
                                        ...     Alguna             Ninguna   
                            Porcentaje  ...   Absoluto Porcentaje Absoluto   
Aconchi                            NaN  ...        NaN        NaN      NaN   
Agua Prieta                        NaN  ...        NaN        NaN      NaN   
Alamos                             NaN  ...        NaN        NaN      NaN   
Altar                              NaN  ...        NaN        NaN      NaN   
Arivechi                           

In [343]:
urban_data = dataframe_tolist(df)
urban2020.iloc[0] = urban_data

for m,mun in enumerate(Municipios):
    df = sonora[mun][14].copy()
    df, extra_urban = format_df(df)
    urban_data = dataframe_tolist(df)
    urban2020.iloc[m] = urban_data

### Entorno Urbano 2020 P2 (15)

In [344]:
df = sonora[mun][15].copy()
df

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,NA
278,Entorno urbano (2020),Absoluto,nan,Porcentaje,nan,2
279,Letrero con nombre de la calle en todas las vi...,11,nan,3.7,nan,2
280,Letrero con nombre de la calle en alguna vialidad,123,nan,41,nan,2
281,Letrero con nombre de la calle en ninguna vial...,166,nan,55.3,nan,2
282,Banqueta en todas las vialidades,0,nan,0,nan,2
283,Banqueta en alguna vialidad,42,nan,14,nan,2
284,Banqueta en ninguna vialidad,258,nan,86,nan,2
285,Transporte colectivo en todas las vialidades,0,nan,0,nan,2
286,Transporte colectivo en alguna vialidad,77,nan,25.7,nan,2
287,Transporte colectivo en ninguna vialidad,223,nan,74.3,nan,2


In [345]:
c0_2 = ['Letrero con nombre de la calle', 'Banqueta', 'Transporte colectivo']
c1 = ['Todas', 'Alguna', 'Ninguna']

u0 = [c0 for c0 in c0_2 for j in range(len(clast) * len(c1))]
u1 = len(c0_2) * ['Todas', 'Todas', 'Alguna', 'Alguna', 'Ninguna', 'Ninguna']
u2 = [c  for j in range(len(c1) * len(c0_2)) for c in clast]

urban_zero = u0
urban_one =  u1
urban_two = u2

mi = pd.MultiIndex.from_arrays([urban_zero, urban_one, urban_two])
urban2020_2 = pd.DataFrame(index = Municipios, columns=mi)
urban2020_2.head()


Letrero con nombre de la calle                                 \
                                     Todas              Alguna              
                                  Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                                NaN        NaN      NaN        NaN   
Agua Prieta                            NaN        NaN      NaN        NaN   
Alamos                                 NaN        NaN      NaN        NaN   
Altar                                  NaN        NaN      NaN        NaN   
Arivechi                               NaN        NaN      NaN        NaN   

                                Banqueta                                 \
             Ninguna               Todas              Alguna              
            Absoluto Porcentaje Absoluto Porcentaje Absoluto Porcentaje   
Aconchi          NaN        NaN      NaN        NaN      NaN        NaN   
Agua Prieta      NaN        NaN      NaN        NaN      NaN        NaN   
Alamos           NaN        NaN      NaN        NaN      NaN        NaN   
Altar            NaN        NaN      NaN        NaN      NaN        NaN   
Arivechi         NaN        NaN      NaN        NaN      NaN        NaN   

                                Transporte colectivo                      \
             Ninguna                           Todas              Alguna   
            Absoluto Porcentaje             Absoluto Porcentaje Absoluto   
Aconchi          NaN        NaN                  NaN        NaN      NaN   
Agua Prieta      NaN        NaN                  NaN        NaN      NaN   
Alamos           NaN        NaN                  NaN        NaN      NaN   
Altar            NaN        NaN                  NaN        NaN      NaN   
Arivechi         NaN        NaN                  NaN        NaN      NaN   

                                            
                        Ninguna             
            Porcentaje Absoluto Porcentaje  
Aconchi            NaN      NaN        NaN  
Agua Prieta        NaN      NaN        NaN  
Alamos             NaN      NaN        NaN  
Altar              NaN      NaN        NaN  
Arivechi           NaN      NaN        NaN

In [346]:

for m,mun in enumerate(Municipios):
    df = sonora[mun][15].copy()
    df, extra_urban = format_df(df)
    urban_data = dataframe_tolist(df)
    urban2020_2.iloc[m] = urban_data

urban_2020 = pd.concat([urban2020, urban2020_2], axis = 1)

save_df(urban_2020, "EntornoUrbano2020.xlsx")


### Infraestructura 2021 (16/17)

In [347]:
df = sonora[mun][16].copy()
df, extra_struct = format_df(df)
df2 = sonora[mun][17].copy()
df2, extra_struct2 = format_df(df2)

df = pd.concat([df, df2])
df
struct_df = pd.DataFrame(index = Municipios, columns= df.index)

for m,mun in enumerate(Municipios):
    df = sonora[mun][16].copy()
    df, extra_struct = format_df(df)
    df2 = sonora[mun][17].copy()
    df2, extra_struct2 = format_df(df2)
    df = pd.concat([df, df2])
    struct_df.iloc[m] = df['Absoluto']

save_df(struct_df, "Estructuras2021.xlsx")

### Vehiculos (18)

In [348]:
df = sonora[mun][18].copy()
df, extra_cars = format_df(df)
df

c0 = list(df.index)
cars_one = [c for c in c0 for j in range(2)]
cars_two = [last for j in range(len(c0))  for last in clast]

mi_cars = pd.MultiIndex.from_arrays([cars_one, cars_two])
cars2022 = pd.DataFrame(index = Municipios, columns= mi_cars) 

for m, mun in enumerate(Municipios):
    df = sonora[mun][18].copy()
    df, extra_cars = format_df(df)
    cars_data = dataframe_tolist(df)
    cars2022.iloc[m] = cars_data

save_df(cars2022, "Automoviles2022.xlsx")


### Accidentes de tránsito (19)


In [349]:
df = sonora[mun][19].copy()
df, extra_acc = format_df(df)

acc0 = list(df.index)
acc_one = [c for c in acc0 for j in range(2)]
acc_two = [last  for j in range(len(acc0)) for last in clast]

mi_acc = pd.MultiIndex.from_arrays([acc_one, acc_two])
acc2022 = pd.DataFrame(index = Municipios, columns= mi_acc) 

for m, mun in enumerate(Municipios):
    df = sonora[mun][19].copy()
    df, extra_acc = format_df(df)
    acc_data = dataframe_tolist(df)
    acc2022.iloc[m] = acc_data

save_df(acc2022, "AccidentesTransito2022.xlsx")


### Seguridad Sep 2023 (20)



In [350]:
df = sonora[mun][20].copy()
df = df.set_index(df.columns[0]).replace({"nan":pd.NA}).dropna(axis = 1, how = 'all')
data = df.iloc[1]

seg_one = [df.index[0],df.index[0]] 

mi_seg = pd.MultiIndex.from_arrays([seg_one, clast])
seg_df = pd.DataFrame(index = Municipios, columns=mi_seg)

for m, mun in enumerate(Municipios):
    df = sonora[mun][20].copy()
    df = df.set_index(df.columns[0]).replace({"nan":pd.NA}).dropna(axis = 1, how = 'all')   
    seg_df.iloc[m] = df.iloc[1,:2]


save_df(seg_df, "SeguridadSep2023.xlsx")

### Economía 2020 (21)


In [351]:
df = sonora[mun][21].copy()
df, extra_economy = format_df(df)

df

360,Absoluto,Porcentaje
Economía (2020),,
Población Económicamente Activa (PEA) 1,6425,-
Tasa de participación económica 1,-,56.7
Tasa de desocupación 1,-,1.5
Población ocupada de 12 años y más por sector 2,5301,100
Primario,3115,58.8
"Minería, industrias manufactureras, electricidad y agua",597,11.3
Construcción,143,2.7
Comercio,619,11.7
Servicios,802,15.1


In [352]:
cols = list(df.index)

economy = {
"type": ['Por Sector', 'Por Nivel de Ingreso (Salarios Minímos)'],
"level": ["Total"," 1","1-2","2-3","3-5", "+5","No recibe ingresos"],
"sector": ["Total", "Primario", "Minería, industrias manufactureras, electricidad y agua", "Construcción", "Comercio", "Servicios"]
}

c0_1 = cols[:3]
c0_3 = cols[-3:]

econ_three_1 = [last  for j in range(len(c0_1)) for last in clast]
econ_three_3 = econ_three_1
econ_two_1 = [""  for j in range(len(c0_1)) for last in clast]
econ_two_3 = econ_two_1
econ_one_1 = econ_two_1
econ_one_3 = econ_two_1
econ_one_2 = ['Por sector' for j in range(len(economy['sector'] * len(clast)))] + ['Por Nivel de Ingreso (Salarios Minimos)' for j in range(len(economy['level'] * len(clast)))]
econ_two_2 =[sec for sec in economy["sector"] for j in range(len(clast)) ] + [lev for lev in economy['level'] for j in range(len(clast))]
econ_three_2 = [last for j in range(len(econ_one_2) // 2) for last in clast]

econ_zero_1 = [c for c in c0_1 for j in range(len(clast))]
econ_zero_3 = [c for c in c0_3 for j in range(len(clast))]
econ_zero_2 = len(econ_two_2) * ['Poblacion Ocupada de 12 años y más']

econ_zero = econ_zero_1 + econ_zero_2 + econ_zero_3
econ_one = econ_one_1 + econ_one_2 + econ_one_3
econ_two = econ_two_1 + econ_two_2 + econ_two_3
econ_three = econ_three_1 + econ_three_2 + econ_three_3
mi_econ = pd.MultiIndex.from_arrays([econ_zero, econ_one, econ_two, econ_three])
econ_df = pd.DataFrame(index = Municipios, columns=mi_econ)
econ_df

Población Económicamente Activa (PEA) 1  \
                                                                      
                                                                      
                                                           Absoluto   
Aconchi                                                         NaN   
Agua Prieta                                                     NaN   
Alamos                                                          NaN   
Altar                                                           NaN   
Arivechi                                                        NaN   
...                                                             ...   
Villa Pesqueira                                                 NaN   
Yécora                                                          NaN   
Gral. Plutarco Elías Calles                                     NaN   
Benito Juárez                                                   NaN   
San Ignacio Río Muerto                                          NaN   

                                       Tasa de participación económica 1  \
                                                                           
                                                                           
                            Porcentaje                          Absoluto   
Aconchi                            NaN                               NaN   
Agua Prieta                        NaN                               NaN   
Alamos                             NaN                               NaN   
Altar                              NaN                               NaN   
Arivechi                           NaN                               NaN   
...                                ...                               ...   
Villa Pesqueira                    NaN                               NaN   
Yécora                             NaN                               NaN   
Gral. Plutarco Elías Calles        NaN                               NaN   
Benito Juárez                      NaN                               NaN   
San Ignacio Río Muerto             NaN                               NaN   

                                       Tasa de desocupación 1             \
                                                                           
                                                                           
                            Porcentaje               Absoluto Porcentaje   
Aconchi                            NaN                    NaN        NaN   
Agua Prieta                        NaN                    NaN        NaN   
Alamos                             NaN                    NaN        NaN   
Altar                              NaN                    NaN        NaN   
Arivechi                           NaN                    NaN        NaN   
...                                ...                    ...        ...   
Villa Pesqueira                    NaN                    NaN        NaN   
Yécora                             NaN                    NaN        NaN   
Gral. Plutarco Elías Calles        NaN                    NaN        NaN   
Benito Juárez                      NaN                    NaN        NaN   
San Ignacio Río Muerto             NaN                    NaN        NaN   

                            Poblacion Ocupada de 12 años y más             \
                                                    Por sector              
                                                         Total              
                                                      Absoluto Porcentaje   
Aconchi                                                    NaN        NaN   
Agua Prieta                                                NaN        NaN   
Alamos                                                     NaN        NaN   
Altar                                                      NaN        NaN   
Arivechi                                                   NaN  

Ahora llenamos la tabla. 

In [353]:
s = dataframe_tolist(df)

econ_df.iloc[0] = s

econ_df.head()

Población Económicamente Activa (PEA) 1             \
                                                                 
                                                                 
                                           Absoluto Porcentaje   
Aconchi                                        6425          -   
Agua Prieta                                     NaN        NaN   
Alamos                                          NaN        NaN   
Altar                                           NaN        NaN   
Arivechi                                        NaN        NaN   

            Tasa de participación económica 1             \
                                                           
                                                           
                                     Absoluto Porcentaje   
Aconchi                                     -       56.7   
Agua Prieta                               NaN        NaN   
Alamos                                    NaN        NaN   
Altar                                     NaN        NaN   
Arivechi                                  NaN        NaN   

            Tasa de desocupación 1             \
                                                
                                                
                          Absoluto Porcentaje   
Aconchi                          -        1.5   
Agua Prieta                    NaN        NaN   
Alamos                         NaN        NaN   
Altar                          NaN        NaN   
Arivechi                       NaN        NaN   

            Poblacion Ocupada de 12 años y más                                 \
                                    Por sector                                  
                                         Total            Primario              
                                      Absoluto Porcentaje Absoluto Porcentaje   
Aconchi                                   5301        100     3115       58.8   
Agua Prieta                                NaN        NaN      NaN        NaN   
Alamos                                     NaN        NaN      NaN        NaN   
Altar                                      NaN        NaN      NaN        NaN   
Arivechi                                   NaN        NaN      NaN        NaN   

             ...                                                     \
             ... Por Nivel de Ingreso (Salarios Minimos)              
             ...                                      +5              
             ...                                Absoluto Porcentaje   
Aconchi      ...                                      60        1.1   
Agua Prieta  ...                                     NaN        NaN   
Alamos       ...                                     NaN        NaN   
Altar        ...                                     NaN        NaN   
Arivechi     ...                                     NaN        NaN   

                                          Unidades económicas 3             \
                                                                             
            No recibe ingresos                                               
                      Absoluto Porcentaje              Absoluto Porcentaje   
Aconchi                     40        0.8                   539        0.5   
Agua Prieta                NaN        NaN                   NaN        NaN   
Alamos                     NaN        NaN                   NaN        NaN   
Altar                      NaN        NaN                   NaN        NaN   
Arivechi                   NaN        NaN                   NaN        NaN   

            Producción Bruta Total (Millones de pesos) 3             \
                                                                      
                                                                      
                                                Absoluto Porcentaje   
Aconchi                                           1042.9        0.2   
Agua P

Ahora, lo aplicamos para todos los municipios.

In [354]:
for m, mun in enumerate(Municipios):
    df = sonora[mun][21].copy()
    df, extra_economy = format_df(df)
    s = dataframe_tolist(df)
    econ_df.iloc[m] = s

save_df(econ_df, "Economia2021.xlsx")

### Finanzas Municipales 2021 (22/23)

In [355]:
df = sonora[mun][22].copy()
ingresos2021, extra_fin1 = format_df(df)

df2 = sonora[mun][23].copy()
egresos2021, extra_fin2 = format_df(df2)

c0 = ingresos2021.index
c1 = [last  for j in range(len(c0)) for last in clast]
ingresos_zero = [c for c in c0 for j in range(len(clast))]

mi_ingresos = pd.MultiIndex.from_arrays([ingresos_zero, c1])
ingresos_df = pd.DataFrame(index = Municipios, columns= mi_ingresos)


c00 = egresos2021.index
c01 = [last  for j in range(len(c00)) for last in clast]
egresos_zero = [c for c in c00 for j in range(len(clast))]

mi_egresos = pd.MultiIndex.from_arrays([egresos_zero, c01])
egresos_df = pd.DataFrame(index = Municipios, columns= mi_egresos)


In [356]:
for m, mun in enumerate(Municipios):
    df = sonora[mun][22].copy()
    ingresos2021, extra_fin1 = format_df(df)
    df2 = sonora[mun][23].copy()
    egresos2021, extra_fin2 = format_df(df2)
    data_egresos = dataframe_tolist(egresos2021)
    data_ingresos = dataframe_tolist(ingresos2021)
    egresos_df.iloc[m]  = data_egresos
    ingresos_df.iloc[m] = data_ingresos

Ahora, vamos a juntarlas

In [357]:
egin = ['Ingresos' for j in range(len(ingresos_df.columns))] + ['Egresos' for j in range(len(egresos_df.columns))]

c0 = ingresos2021.index
c1 = [last  for j in range(len(c0)) for last in clast]
ingresos_zero = [c for c in c0 for j in range(len(clast))]

mi_egin = pd.MultiIndex.from_arrays([egin,ingresos_zero + egresos_zero, c1 + c01]) 

egin = pd.DataFrame(index = Municipios, columns = mi_egin)

egin['Egresos'] = egresos_df
egin['Ingresos'] = ingresos_df


save_df(egin, "FinanzasMunicipales2021.xlsx")